<a href="https://colab.research.google.com/github/teaser003838/system-prompts-and-models-of-ai-tools/blob/main/zai_deployment_v1_fixing_packages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===== SYSTEM REQUIREMENTS AND PREREQUISITES =====
from IPython.display import HTML, display, clear_output
import time
import sys
import os
import subprocess
import threading
import json
from datetime import datetime

# Enhanced UI Styles with bright, soothing colors
CSS = """
<style>
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700&display=swap');
body {
    font-family: 'Inter', sans-serif;
    background: linear-gradient(135deg, #f5f7fa 0%, #e4e8f0 100%);
    margin: 0;
    padding: 0;
}
.container {
    max-width: 1000px;
    margin: 0 auto;
    padding: 20px;
}
.title {
    color: #2c3e50;
    font-size: 28px;
    font-weight: 700;
    margin-bottom: 20px;
    text-align: center;
    background: linear-gradient(90deg, #3498db, #2ecc71);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    text-shadow: 0px 2px 4px rgba(0,0,0,0.1);
}
.subtitle {
    color: #34495e;
    font-size: 20px;
    font-weight: 600;
    margin-top: 20px;
    margin-bottom: 15px;
    padding-bottom: 8px;
    border-bottom: 2px solid #3498db;
}
.card {
    background: white;
    border-radius: 12px;
    padding: 20px;
    margin-bottom: 20px;
    box-shadow: 0 4px 15px rgba(0,0,0,0.05);
    transition: transform 0.3s ease, box-shadow 0.3s ease;
}
.card:hover {
    transform: translateY(-5px);
    box-shadow: 0 8px 25px rgba(0,0,0,0.1);
}
.info {
    color: #34495e;
    margin-bottom: 12px;
    font-size: 16px;
    line-height: 1.5;
}
.highlight {
    color: #3498db;
    font-weight: 600;
}
.success {
    color: #2ecc71;
    font-weight: 600;
}
.error {
    color: #e74c3c;
    font-weight: 600;
}
.warning {
    color: #f39c12;
    font-weight: 600;
}
.progress-container {
    background: #ecf0f1;
    border-radius: 10px;
    height: 24px;
    margin: 15px 0;
    overflow: hidden;
    position: relative;
}
.progress-bar {
    background: linear-gradient(90deg, #3498db, #2ecc71);
    height: 100%;
    width: 0%;
    border-radius: 10px;
    transition: width 0.5s ease;
    display: flex;
    align-items: center;
    justify-content: center;
    color: white;
    font-weight: 600;
    font-size: 14px;
}
.status {
    color: #2c3e50;
    font-weight: 600;
    margin-top: 15px;
    text-align: center;
    font-size: 18px;
}
.log-container {
    background: #2c3e50;
    color: #ecf0f1;
    border-radius: 8px;
    padding: 15px;
    margin-top: 20px;
    font-family: 'Courier New', monospace;
    font-size: 14px;
    max-height: 300px;
    overflow-y: auto;
    white-space: pre-wrap;
}
.log-entry {
    margin-bottom: 5px;
}
.log-info {
    color: #3498db;
}
.log-success {
    color: #2ecc71;
}
.log-error {
    color: #e74c3c;
}
.log-warning {
    color: #f39c12;
}
.button {
    background: linear-gradient(90deg, #3498db, #2ecc71);
    color: white;
    border: none;
    padding: 10px 20px;
    border-radius: 6px;
    cursor: pointer;
    font-weight: 600;
    transition: all 0.3s ease;
    margin: 5px;
}
.button:hover {
    transform: translateY(-2px);
    box-shadow: 0 4px 8px rgba(0,0,0,0.2);
}
.model-card {
    background: linear-gradient(135deg, #3498db, #2ecc71);
    color: white;
    border-radius: 12px;
    padding: 15px;
    margin-bottom: 15px;
    box-shadow: 0 4px 15px rgba(0,0,0,0.1);
}
.model-name {
    font-size: 18px;
    font-weight: 600;
    margin-bottom: 5px;
}
.model-desc {
    font-size: 14px;
    opacity: 0.9;
}
</style>
"""

display(HTML(CSS))

# Global variables for logging
log_entries = []
log_lock = threading.Lock()

def add_log_entry(message, log_type="info"):
    timestamp = datetime.now().strftime("%H:%M:%S.%f")[:-3]
    with log_lock:
        log_entries.append(f"[{timestamp}] {message}")
        # Keep only the last 100 log entries
        if len(log_entries) > 100:
            log_entries.pop(0)

    # Update log display
    log_class = f"log-{log_type}"
    log_html = f'<div class="log-entry {log_class}">[{timestamp}] {message}</div>'
    display(HTML(log_html))

def update_progress(percent, status_text=""):
    progress_html = f"""
    <div class="progress-container">
        <div class="progress-bar" style="width: {percent}%">
            {percent}%
        </div>
    </div>
    <div class="status">{status_text}</div>
    """
    display(HTML(progress_html))

def show_status(message, status_type="info"):
    if status_type == "success":
        display(HTML(f'<div class="success">✓ {message}</div>'))
    elif status_type == "error":
        display(HTML(f'<div class="error">✗ {message}</div>'))
    elif status_type == "warning":
        display(HTML(f'<div class="warning">⚠ {message}</div>'))
    else:
        display(HTML(f'<div class="info">• {message}</div>'))

    add_log_entry(message, status_type)

# Initialize
display(HTML('<div class="container">'))
display(HTML('<div class="title">ComfyUI Installation System</div>'))
display(HTML('<div class="card">'))
display(HTML('<div class="subtitle">System Requirements and Prerequisites</div>'))

# Display information with enhanced UI
show_status("This system will install ComfyUI with all dependencies in a Google Colab environment.", "info")
show_status("All packages will be locked to specific versions to prevent automatic upgrades.", "info")

display(HTML('<div class="subtitle">Required Environment:</div>'))
display(HTML('<div class="info">• <span class="highlight">Google Colab</span> with T4 GPU runtime (minimum)</div>'))
display(HTML('<div class="info">• <span class="highlight">Stable internet connection</span> for model downloads</div>'))
display(HTML('<div class="info">• <span class="highlight">Local storage</span> (no Google Drive integration)</div>'))

display(HTML('<div class="subtitle">Models to be Installed:</div>'))
display(HTML('<div class="model-card">'))
display(HTML('<div class="model-name">RealVisXL_V4_Lightning</div>'))
display(HTML('<div class="model-desc">Photorealistic SDXL model optimized for faster generation</div>'))
display(HTML('</div>'))

display(HTML('<div class="model-card">'))
display(HTML('<div class="model-name">JuggernautXL_v9</div>'))
display(HTML('<div class="model-desc">Versatile SDXL model good for various styles and subjects</div>'))
display(HTML('</div>'))

display(HTML('<div class="model-card">'))
display(HTML('<div class="model-name">epiCRealismXL</div>'))
display(HTML('<div class="model-desc">Photorealistic SDXL model focused on realistic image generation</div>'))
display(HTML('</div>'))

display(HTML('<div class="subtitle">What This System Provides:</div>'))
display(HTML('<div class="info">• Step-by-step installation with specific package versions tested to work together</div>'))
display(HTML('<div class="info">• Real-time verification of installations and downloads</div>'))
display(HTML('<div class="info">• Interactive progress tracking with detailed logging</div>'))
display(HTML('<div class="info">• Comprehensive error prevention and troubleshooting</div>'))
display(HTML('<div class="info">• Package version locking to prevent automatic upgrades</div>'))

show_status("Ready to Begin", "success")

# Create a simple summary report
summary = {
    "status": "Informational",
    "details": "System requirements and prerequisites outlined",
    "next_step": "Proceed to Step 1: Local Directory Setup"
}

display(HTML('<div class="subtitle">Summary Report</div>'))
display(HTML(f'<div class="info">Status: {summary["status"]}</div>'))
display(HTML(f'<div class="info">Details: {summary["details"]}</div>'))
display(HTML(f'<div class="info">Next Step: {summary["next_step"]}</div>'))

display(HTML('</div>'))  # Close card
display(HTML('</div>'))  # Close container

# Print summary report
print("\n" + "="*50)
print("SUMMARY REPORT")
print("="*50)
print(f"Status: {summary['status']}")
print(f"Details: {summary['details']}")
print(f"Next Step: {summary['next_step']}")
print("="*50)

In [ ]:
# ===== STEP 1: LOCAL DIRECTORY SETUP =====
from IPython.display import HTML, display, clear_output
import os
import time
import threading
import shutil

# Reuse CSS from previous cell
display(HTML(CSS))

# Initialize
display(HTML('<div class="container">'))
display(HTML('<div class="title">Local Directory Setup</div>'))
display(HTML('<div class="card">'))

update_progress(0, "Initializing directory setup...")
add_log_entry("Starting directory setup process", "info")

# Track created directories
created_dirs = []
existing_dirs = []
errors = []

# List of directories to create
required_dirs = [
    '/content/ComfyUI_Models',
    '/content/ComfyUI_Outputs',
    '/content/ComfyUI_Models/checkpoints',
    '/content/ComfyUI_Models/loras',
    '/content/ComfyUI_Models/loras/face_enhancement',
    '/content/ComfyUI_Models/loras/lighting_photography',
    '/content/ComfyUI_Models/loras/style_effects',
    '/content/ComfyUI_Models/loras/detail_boost',
    '/content/ComfyUI_Models/controlnet',
    '/content/ComfyUI_Models/upscale_models',
    '/content/ComfyUI_Models/vae'
]

total_dirs = len(required_dirs)
created_count = 0

# Create directories
for i, dir_path in enumerate(required_dirs):
    progress = int((i / total_dirs) * 100)
    update_progress(progress, f"Processing: {dir_path}")
    add_log_entry(f"Processing directory: {dir_path}", "info")

    if os.path.exists(dir_path):
        existing_dirs.append(dir_path)
        show_status(f"Directory already exists: {dir_path}", "info")
        add_log_entry(f"Directory already exists: {dir_path}", "info")
    else:
        try:
            os.makedirs(dir_path, exist_ok=True)
            created_dirs.append(dir_path)
            show_status(f"Created directory: {dir_path}", "success")
            add_log_entry(f"Created directory: {dir_path}", "success")
            created_count += 1
        except Exception as e:
            errors.append(f"Error creating directory {dir_path}: {str(e)}")
            show_status(f"Error creating directory {dir_path}: {str(e)}", "error")
            add_log_entry(f"Error creating directory {dir_path}: {str(e)}", "error")

    time.sleep(0.2)  # Small delay for visual feedback

# Final progress update
update_progress(100, "Directory setup completed!")
add_log_entry("Directory setup process completed", "success")

# Display directory structure
display(HTML('<div class="subtitle">Directory Structure</div>'))
display(HTML('<div class="info">/content/ComfyUI_Models/</div>'))
display(HTML('<div class="info">├── checkpoints/</div>'))
display(HTML('<div class="info">├── loras/</div>'))
display(HTML('<div class="info">&nbsp;&nbsp;├── face_enhancement/</div>'))
display(HTML('<div class="info">&nbsp;&nbsp;├── lighting_photography/</div>'))
display(HTML('<div class="info">&nbsp;&nbsp;├── style_effects/</div>'))
display(HTML('<div class="info">&nbsp;&nbsp;└── detail_boost/</div>'))
display(HTML('<div class="info">├── controlnet/</div>'))
display(HTML('<div class="info">├── upscale_models/</div>'))
display(HTML('<div class="info">└── vae/</div>'))
display(HTML('<div class="info">/content/ComfyUI_Outputs/</div>'))

# Summary report
display(HTML('<div class="subtitle">Directory Setup Summary</div>'))
display(HTML(f'<div class="info">• Total directories processed: {total_dirs}</div>'))
display(HTML(f'<div class="info">• New directories created: {len(created_dirs)}</div>'))
display(HTML(f'<div class="info">• Existing directories found: {len(existing_dirs)}</div>'))
display(HTML(f'<div class="info">• Errors encountered: {len(errors)}</div>'))

if errors:
    display(HTML('<div class="error">Some errors occurred during directory setup</div>'))
    for error in errors:
        display(HTML(f'<div class="error">{error}</div>'))
else:
    display(HTML('<div class="success">Directory structure is ready for ComfyUI installation</div>'))

display(HTML('</div>'))  # Close card
display(HTML('</div>'))  # Close container

# Print summary report
print("\n" + "="*50)
print("SUMMARY REPORT")
print("="*50)
print(f"Status: Directory Setup Completed")
print(f"Total Directories: {total_dirs}")
print(f"Created: {len(created_dirs)}")
print(f"Already Existed: {len(existing_dirs)}")
print(f"Errors: {len(errors)}")
if errors:
    print("\nErrors:")
    for error in errors:
        print(f"  - {error}")
print(f"\nNext Step: Uninstall Existing Packages")
print("="*50)

In [ ]:
# ===== STEP 2: UNINSTALL EXISTING PACKAGES =====
from IPython.display import HTML, display, clear_output
import subprocess
import sys
import os
import time
import importlib
import threading

# Reuse CSS from previous cell
display(HTML(CSS))

# Function to uninstall package
def uninstall_package(package_name):
    try:
        result = subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', package_name],
                              capture_output=True, text=True)
        if result.returncode == 0:
            show_status(f"Successfully uninstalled: {package_name}", "success")
            add_log_entry(f"Successfully uninstalled: {package_name}", "success")
            return True
        else:
            show_status(f"Failed to uninstall {package_name}: {result.stderr}", "error")
            add_log_entry(f"Failed to uninstall {package_name}: {result.stderr}", "error")
            return False
    except Exception as e:
        show_status(f"Error uninstalling {package_name}: {str(e)}", "error")
        add_log_entry(f"Error uninstalling {package_name}: {str(e)}", "error")
        return False

# Initialize
display(HTML('<div class="container">'))
display(HTML('<div class="title">Uninstall Existing Packages</div>'))
display(HTML('<div class="card">'))

update_progress(0, "Initializing package uninstallation...")
add_log_entry("Starting package uninstallation process", "info")

# Define packages to uninstall
packages_to_uninstall = [
    'torch', 'torchvision', 'torchaudio', 'xformers',
    'numpy', 'scipy', 'scikit-image', 'opencv-python',
    'opencv-contrib-python', 'pillow', 'matplotlib',
    'transformers', 'diffusers', 'accelerate', 'safetensors',
    'albumentations', 'ultralytics', 'insightface',
    'onnxruntime-gpu', 'gradio'
]

total_packages = len(packages_to_uninstall)
uninstalled_packages = []
failed_uninstalls = []
not_installed_packages = []

# Uninstall packages
for i, package in enumerate(packages_to_uninstall):
    progress = int((i / total_packages) * 100)
    update_progress(progress, f"Processing: {package}")
    add_log_entry(f"Processing package: {package}", "info")

    # Check if package is installed
    try:
        spec = importlib.util.find_spec(package)
        if spec is None:
            not_installed_packages.append(package)
            show_status(f"Package not installed: {package}", "info")
            add_log_entry(f"Package not installed: {package}", "info")
        else:
            success = uninstall_package(package)
            if success:
                uninstalled_packages.append(package)
            else:
                failed_uninstalls.append(package)
    except Exception as e:
        not_installed_packages.append(package)
        show_status(f"Error checking package {package}: {str(e)}", "warning")
        add_log_entry(f"Error checking package {package}: {str(e)}", "warning")

    time.sleep(0.3)  # Small delay for visual feedback

# Final progress update
update_progress(100, "Package uninstallation completed!")
add_log_entry("Package uninstallation process completed", "success")

# Summary report
display(HTML('<div class="subtitle">Package Uninstallation Summary</div>'))
display(HTML(f'<div class="info">• Total packages processed: {total_packages}</div>'))
display(HTML(f'<div class="info">• Successfully uninstalled: {len(uninstalled_packages)}</div>'))
display(HTML(f'<div class="info">• Not installed: {len(not_installed_packages)}</div>'))
display(HTML(f'<div class="info">• Failed to uninstall: {len(failed_uninstalls)}</div>'))

if failed_uninstalls:
    display(HTML('<div class="error">Some packages could not be uninstalled</div>'))
    for pkg in failed_uninstalls:
        display(HTML(f'<div class="error">- {pkg}</div>'))
else:
    display(HTML('<div class="success">All packages processed successfully</div>'))

display(HTML('</div>'))  # Close card
display(HTML('</div>'))  # Close container

# Print summary report
print("\n" + "="*50)
print("SUMMARY REPORT")
print("="*50)
print(f"Status: Package Uninstallation")
print(f"Total Packages: {total_packages}")
print(f"Uninstalled: {len(uninstalled_packages)}")
print(f"Not Installed: {len(not_installed_packages)}")
print(f"Failed: {len(failed_uninstalls)}")

if uninstalled_packages:
    print("\nUninstalled Packages:")
    for pkg in uninstalled_packages:
        print(f"  - {pkg}")

if failed_uninstalls:
    print("\nFailed Uninstalls:")
    for pkg in failed_uninstalls:
        print(f"  - {pkg}")

print(f"\nNext Step: Foundation Package Installation")
print("="*50)

In [ ]:
# ===== STEP 3: FOUNDATION PACKAGE INSTALLATION =====
from IPython.display import HTML, display, clear_output
import subprocess
import sys
import os
import time
import importlib
import threading

# Reuse CSS from previous cell
display(HTML(CSS))

# Function to install package with version locking
def install_package(package_spec, force_reinstall=False, index_url=None):
    package_name = package_spec.split('==')[0] if '==' in package_spec else package_spec

    # Check if already installed with correct version
    if not force_reinstall:
        if '==' in package_spec:
            pkg_name, required_version = package_spec.split('==')
            try:
                result = subprocess.run([sys.executable, '-m', 'pip', 'show', pkg_name],
                                      capture_output=True, text=True)
                if result.returncode == 0:
                    for line in result.stdout.split('\n'):
                        if line.startswith('Version:'):
                            installed_version = line.split(':')[1].strip()
                            if installed_version == required_version:
                                show_status(f"Package already installed with correct version: {package_spec}", "info")
                                add_log_entry(f"Package already installed with correct version: {package_spec}", "info")
                                return True
            except Exception:
                pass

    # Install package with version locking
    cmd = [sys.executable, '-m', 'pip', 'install']

    # Add index URL if specified
    if index_url:
        cmd.extend(['--index-url', index_url])

    # Add flags to prevent upgrades
    cmd.extend(['--upgrade-strategy', 'only-if-needed'])

    if force_reinstall:
        cmd.extend(['--force-reinstall', '--no-cache-dir'])

    cmd.append(package_spec)

    try:
        show_status(f"Installing: {package_spec}", "info")
        add_log_entry(f"Installing package: {package_spec}", "info")

        result = subprocess.run(cmd, capture_output=True, text=True)
        if result.returncode == 0:
            show_status(f"Successfully installed: {package_spec}", "success")
            add_log_entry(f"Successfully installed: {package_spec}", "success")
            return True
        else:
            show_status(f"Failed to install {package_spec}: {result.stderr}", "error")
            add_log_entry(f"Failed to install {package_spec}: {result.stderr}", "error")
            return False
    except Exception as e:
        show_status(f"Error installing {package_spec}: {str(e)}", "error")
        add_log_entry(f"Error installing {package_spec}: {str(e)}", "error")
        return False

# Initialize
display(HTML('<div class="container">'))
display(HTML('<div class="title">Foundation Package Installation</div>'))
display(HTML('<div class="card">'))

update_progress(0, "Initializing package installation...")
add_log_entry("Starting foundation package installation process", "info")

# Define packages to install with specific versions
packages = [
    {"name": "numpy==1.26.2", "description": "Numerical computing foundation", "force": True},
    {"name": "torch==2.1.2+cu121", "description": "PyTorch with CUDA 12.1", "force": True, "index_url": "https://download.pytorch.org/whl/cu121"},
    {"name": "torchvision==0.16.2+cu121", "description": "Vision utilities for PyTorch", "force": True, "index_url": "https://download.pytorch.org/whl/cu121"},
    {"name": "torchaudio==2.1.2+cu121", "description": "Audio utilities for PyTorch", "force": True, "index_url": "https://download.pytorch.org/whl/cu121"},
    {"name": "xformers==0.0.23", "description": "Memory optimization for transformers", "force": True}
]

total_packages = len(packages)
installed_packages = []
skipped_packages = []
failed_packages = []

# Install packages
for i, package in enumerate(packages):
    progress = int((i / total_packages) * 100)
    update_progress(progress, f"Installing {package['name']} ({package['description']})")
    add_log_entry(f"Installing {package['name']} ({package['description']})", "info")

    # Install package with index URL if specified
    index_url = package.get('index_url', None)
    success = install_package(package['name'], package.get('force', False), index_url)

    if success:
        installed_packages.append(package['name'])
    else:
        failed_packages.append(package['name'])

    time.sleep(0.5)  # Small delay for visual feedback

# Final progress update
update_progress(100, "Package installation completed!")
add_log_entry("Foundation package installation process completed", "success")

# Summary report
display(HTML('<div class="subtitle">Package Installation Summary</div>'))
display(HTML(f'<div class="info">• Total packages processed: {total_packages}</div>'))
display(HTML(f'<div class="info">• Successfully installed: {len(installed_packages)}</div>'))
display(HTML(f'<div class="info">• Skipped (already installed): {len(skipped_packages)}</div>'))
display(HTML(f'<div class="info">• Failed installations: {len(failed_packages)}</div>'))

if failed_packages:
    display(HTML('<div class="error">Some packages failed to install</div>'))
    for pkg in failed_packages:
        display(HTML(f'<div class="error">- {pkg}</div>'))
else:
    display(HTML('<div class="success">All packages installed successfully</div>'))

display(HTML('</div>'))  # Close card
display(HTML('</div>'))  # Close container

# Print summary report
print("\n" + "="*50)
print("SUMMARY REPORT")
print("="*50)
print(f"Status: Foundation Package Installation")
print(f"Total Packages: {total_packages}")
print(f"Successfully Installed: {len(installed_packages)}")
print(f"Skipped: {len(skipped_packages)}")
print(f"Failed: {len(failed_packages)}")

if installed_packages:
    print("\nInstalled Packages:")
    for pkg in installed_packages:
        print(f"  - {pkg}")

if skipped_packages:
    print("\nSkipped Packages:")
    for pkg in skipped_packages:
        print(f"  - {pkg}")

if failed_packages:
    print("\nFailed Packages:")
    for pkg in failed_packages:
        print(f"  - {pkg}")

print(f"\nNext Step: Install All Dependencies")
print("="*50)

In [ ]:
# ===== STEP 4: INSTALL ALL DEPENDENCIES =====
from IPython.display import HTML, display, clear_output
import subprocess
import sys
import os
import time
import importlib
import threading
from datetime import datetime

# Define colored font styles
CSS = """
<style>
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700&display=swap');
body {
    font-family: 'Inter', sans-serif;
    background: linear-gradient(135deg, #f5f7fa 0%, #e4e8f0 100%);
    margin: 0;
    padding: 0;
}
.container {
    max-width: 1000px;
    margin: 0 auto;
    padding: 20px;
}
.title {
    color: #2c3e50;
    font-size: 28px;
    font-weight: 700;
    margin-bottom: 20px;
    text-align: center;
    background: linear-gradient(90deg, #3498db, #2ecc71);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    text-shadow: 0px 2px 4px rgba(0,0,0,0.1);
}
.subtitle {
    color: #34495e;
    font-size: 20px;
    font-weight: 600;
    margin-top: 20px;
    margin-bottom: 15px;
    padding-bottom: 8px;
    border-bottom: 2px solid #3498db;
}
.card {
    background: white;
    border-radius: 12px;
    padding: 20px;
    margin-bottom: 20px;
    box-shadow: 0 4px 15px rgba(0,0,0,0.05);
    transition: transform 0.3s ease, box-shadow 0.3s ease;
}
.card:hover {
    transform: translateY(-5px);
    box-shadow: 0 8px 25px rgba(0,0,0,0.1);
}
.info {
    color: #34495e;
    margin-bottom: 12px;
    font-size: 16px;
    line-height: 1.5;
}
.highlight {
    color: #3498db;
    font-weight: 600;
}
.success {
    color: #2ecc71;
    font-weight: 600;
}
.error {
    color: #e74c3c;
    font-weight: 600;
}
.warning {
    color: #f39c12;
    font-weight: 600;
}
.progress-container {
    background: #ecf0f1;
    border-radius: 10px;
    height: 24px;
    margin: 15px 0;
    overflow: hidden;
    position: relative;
}
.progress-bar {
    background: linear-gradient(90deg, #3498db, #2ecc71);
    height: 100%;
    width: 0%;
    border-radius: 10px;
    transition: width 0.5s ease;
    display: flex;
    align-items: center;
    justify-content: center;
    color: white;
    font-weight: 600;
    font-size: 14px;
}
.status {
    color: #2c3e50;
    font-weight: 600;
    margin-top: 15px;
    text-align: center;
    font-size: 18px;
}
.log-container {
    background: #2c3e50;
    color: #ecf0f1;
    border-radius: 8px;
    padding: 15px;
    margin-top: 20px;
    font-family: 'Courier New', monospace;
    font-size: 14px;
    max-height: 300px;
    overflow-y: auto;
    white-space: pre-wrap;
}
.log-entry {
    margin-bottom: 5px;
}
.log-info {
    color: #3498db;
}
.log-success {
    color: #2ecc71;
}
.log-error {
    color: #e74c3c;
}
.log-warning {
    color: #f39c12;
}
.button {
    background: linear-gradient(90deg, #3498db, #2ecc71);
    color: white;
    border: none;
    padding: 10px 20px;
    border-radius: 6px;
    cursor: pointer;
    font-weight: 600;
    transition: all 0.3s ease;
    margin: 5px;
}
.button:hover {
    transform: translateY(-2px);
    box-shadow: 0 4px 8px rgba(0,0,0,0.2);
}
.model-card {
    background: linear-gradient(135deg, #3498db, #2ecc71);
    color: white;
    border-radius: 12px;
    padding: 15px;
    margin-bottom: 15px;
    box-shadow: 0 4px 15px rgba(0,0,0,0.1);
}
.model-name {
    font-size: 18px;
    font-weight: 600;
    margin-bottom: 5px;
}
.model-desc {
    font-size: 14px;
    opacity: 0.9;
}
</style>
"""

# Global variables for logging
log_entries = []
log_lock = threading.Lock()

def add_log_entry(message, log_type="info"):
    timestamp = datetime.now().strftime("%H:%M:%S.%f")[:-3]
    with log_lock:
        log_entries.append(f"[{timestamp}] {message}")
        # Keep only the last 100 log entries
        if len(log_entries) > 100:
            log_entries.pop(0)

    # Update log display
    log_class = f"log-{log_type}"
    log_html = f'<div class="log-entry {log_class}">[{timestamp}] {message}</div>'
    display(HTML(log_html))

def update_progress(percent, status_text=""):
    progress_html = f"""
    <div class="progress-container">
        <div class="progress-bar" style="width: {percent}%">
            {percent}%
        </div>
    </div>
    <div class="status">{status_text}</div>
    """
    display(HTML(progress_html))

def show_status(message, status_type="info"):
    if status_type == "success":
        display(HTML(f'<div class="success">✓ {message}</div>'))
    elif status_type == "error":
        display(HTML(f'<div class="error">✗ {message}</div>'))
    elif status_type == "warning":
        display(HTML(f'<div class="warning">⚠ {message}</div>'))
    else:
        display(HTML(f'<div class="info">• {message}</div>'))

    add_log_entry(message, status_type)

# Function to install package with version locking
def install_package(package_spec, force_reinstall=False):
    # Ensure package_spec is not None
    if package_spec is None:
        show_status("Package specification is None", "error")
        add_log_entry("Package specification is None", "error")
        return False

    # Extract package name
    if '==' in package_spec:
        package_name = package_spec.split('==')[0]
    elif '>=' in package_spec:
        package_name = package_spec.split('>=')[0]
    else:
        package_name = package_spec

    # Check if already installed with correct version
    if not force_reinstall:
        if '==' in package_spec:
            pkg_name, required_version = package_spec.split('==')
            try:
                result = subprocess.run([sys.executable, '-m', 'pip', 'show', pkg_name],
                                      capture_output=True, text=True)
                if result.returncode == 0:
                    for line in result.stdout.split('\n'):
                        if line.startswith('Version:'):
                            installed_version = line.split(':')[1].strip()
                            if installed_version == required_version:
                                show_status(f"Package already installed with correct version: {package_spec}", "info")
                                add_log_entry(f"Package already installed with correct version: {package_spec}", "info")
                                return True
            except Exception:
                pass
        elif is_package_installed(package_name):
            show_status(f"Package already installed: {package_name}", "info")
            add_log_entry(f"Package already installed: {package_name}", "info")
            return True

    # Install package with version locking
    cmd = [sys.executable, '-m', 'pip', 'install']

    # Add flags to prevent upgrades
    cmd.extend(['--upgrade-strategy', 'only-if-needed'])

    if force_reinstall:
        cmd.extend(['--force-reinstall', '--no-cache-dir'])

    cmd.append(package_spec)

    try:
        show_status(f"Installing: {package_spec}", "info")
        add_log_entry(f"Installing package: {package_spec}", "info")

        result = subprocess.run(cmd, capture_output=True, text=True)
        if result.returncode == 0:
            show_status(f"Successfully installed: {package_spec}", "success")
            add_log_entry(f"Successfully installed: {package_spec}", "success")
            return True
        else:
            show_status(f"Failed to install {package_spec}: {result.stderr}", "error")
            add_log_entry(f"Failed to install {package_spec}: {result.stderr}", "error")
            return False
    except Exception as e:
        show_status(f"Error installing {package_spec}: {str(e)}", "error")
        add_log_entry(f"Error installing {package_spec}: {str(e)}", "error")
        return False

# Function to check if package is installed
def is_package_installed(package_name):
    try:
        spec = importlib.util.find_spec(package_name)
        return spec is not None
    except Exception:
        return False

# Function to install package group
def install_package_group(category_name, packages):
    display(HTML(f'<div class="subtitle">{category_name}</div>'))
    add_log_entry(f"Installing package group: {category_name}", "info")

    installed = []
    skipped = []
    failed = []

    for package in packages:
        # Skip None packages
        if package is None:
            continue

        # Check if already installed with correct version
        if '==' in package:
            pkg_name = package.split('==')[0]
            required_version = package.split('==')[1]
        elif '>=' in package:
            pkg_name = package.split('>=')[0]
            required_version = None
        else:
            pkg_name = package
            required_version = None

        if is_package_installed(pkg_name):
            skipped.append(package)
            show_status(f"Skipping already installed package: {package}", "info")
            add_log_entry(f"Skipping already installed package: {package}", "info")
        else:
            # Install package
            success = install_package(package, force_reinstall=True)

            if success:
                installed.append(package)
            else:
                failed.append(package)

        time.sleep(0.3)  # Small delay for visual feedback

    return installed, skipped, failed

# Initialize
display(HTML(CSS))
display(HTML('<div class="container">'))
display(HTML('<div class="title">Install All Dependencies</div>'))
display(HTML('<div class="card">'))

update_progress(0, "Initializing dependency installation...")
add_log_entry("Starting dependency installation process", "info")

# Define package groups
package_groups = [
    {
        "name": "Scientific Computing Stack",
        "packages": [
            "scipy==1.13.0",
            "scikit-image==0.22.0",
            "matplotlib==3.8.2"
        ]
    },
    {
        "name": "Image Processing",
        "packages": [
            "opencv-python==4.8.0.76",
            "opencv-contrib-python==4.8.0.76",
            "pillow==10.2.0"
        ]
    },
    {
        "name": "Machine Learning Libraries",
        "packages": [
            "transformers==4.36.2",
            "diffusers==0.25.0",
            "accelerate==0.25.0",
            "safetensors==0.4.1"
        ]
    },
    {
        "name": "Image Augmentation",
        "packages": [
            "albumentations==1.3.1"
        ]
    },
    {
        "name": "Detection and Face Analysis",
        "packages": [
            "ultralytics==8.0.232",
            "insightface==0.7.3",
            "onnx>=1.15.0",
            "opencv-python>=4.8.0",
            "scikit-image>=0.22.0",
            "Cython"
        ]
    },
    {
        "name": "Web Interface",
        "packages": [
            "gradio==4.8.0"
        ]
    }
]

# Track installation results
all_installed = []
all_skipped = []
all_failed = []
total_groups = len(package_groups)

# Install package groups
for i, group in enumerate(package_groups):
    progress = int((i / total_groups) * 100)
    update_progress(progress, f"Installing {group['name']} packages...")
    add_log_entry(f"Installing package group: {group['name']}", "info")

    installed, skipped, failed = install_package_group(group['name'], group['packages'])
    all_installed.extend(installed)
    all_skipped.extend(skipped)
    all_failed.extend(failed)

    time.sleep(0.5)  # Small delay for visual feedback

# Final progress update
update_progress(100, "Dependency installation completed!")
add_log_entry("Dependency installation process completed", "success")

# Summary report
display(HTML('<div class="subtitle">Dependency Installation Summary</div>'))
display(HTML(f'<div class="info">• Total package groups processed: {total_groups}</div>'))
display(HTML(f'<div class="info">• Successfully installed packages: {len(all_installed)}</div>'))
display(HTML(f'<div class="info">• Skipped (already installed): {len(all_skipped)}</div>'))
display(HTML(f'<div class="info">• Failed installations: {len(all_failed)}</div>'))

if all_failed:
    display(HTML('<div class="error">Some packages failed to install</div>'))
    for pkg in all_failed:
        display(HTML(f'<div class="error">- {pkg}</div>'))
else:
    display(HTML('<div class="success">All dependencies installed successfully</div>'))

display(HTML('</div>'))  # Close card
display(HTML('</div>'))  # Close container

# Print summary report
print("\n" + "="*50)
print("SUMMARY REPORT")
print("="*50)
print(f"Status: Dependency Installation")
print(f"Total Package Groups: {total_groups}")
print(f"Successfully Installed: {len(all_installed)}")
print(f"Skipped: {len(all_skipped)}")
print(f"Failed: {len(all_failed)}")

if all_installed:
    print("\nInstalled Packages:")
    for pkg in all_installed:
        print(f"  - {pkg}")

if all_skipped:
    print("\nSkipped Packages:")
    for pkg in all_skipped:
        print(f"  - {pkg}")

if all_failed:
    print("\nFailed Packages:")
    for pkg in all_failed:
        print(f"  - {pkg}")

print(f"\nNext Step: Clone and Configure ComfyUI")
print("="*50)

In [ ]:
# ===== STEP 5: CLONE AND CONFIGURE COMFYUI =====
from IPython.display import HTML, display, clear_output
import subprocess
import sys
import os
import time
import shutil
import threading
from datetime import datetime

# Define colored font styles
CSS = """
<style>
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700&display=swap');
body {
    font-family: 'Inter', sans-serif;
    background: linear-gradient(135deg, #f5f7fa 0%, #e4e8f0 100%);
    margin: 0;
    padding: 0;
}
.container {
    max-width: 1000px;
    margin: 0 auto;
    padding: 20px;
}
.title {
    color: #2c3e50;
    font-size: 28px;
    font-weight: 700;
    margin-bottom: 20px;
    text-align: center;
    background: linear-gradient(90deg, #3498db, #2ecc71);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    text-shadow: 0px 2px 4px rgba(0,0,0,0.1);
}
.subtitle {
    color: #34495e;
    font-size: 20px;
    font-weight: 600;
    margin-top: 20px;
    margin-bottom: 15px;
    padding-bottom: 8px;
    border-bottom: 2px solid #3498db;
}
.card {
    background: white;
    border-radius: 12px;
    padding: 20px;
    margin-bottom: 20px;
    box-shadow: 0 4px 15px rgba(0,0,0,0.05);
    transition: transform 0.3s ease, box-shadow 0.3s ease;
}
.card:hover {
    transform: translateY(-5px);
    box-shadow: 0 8px 25px rgba(0,0,0,0.1);
}
.info {
    color: #34495e;
    margin-bottom: 12px;
    font-size: 16px;
    line-height: 1.5;
}
.highlight {
    color: #3498db;
    font-weight: 600;
}
.success {
    color: #2ecc71;
    font-weight: 600;
}
.error {
    color: #e74c3c;
    font-weight: 600;
}
.warning {
    color: #f39c12;
    font-weight: 600;
}
.progress-container {
    background: #ecf0f1;
    border-radius: 10px;
    height: 24px;
    margin: 15px 0;
    overflow: hidden;
    position: relative;
}
.progress-bar {
    background: linear-gradient(90deg, #3498db, #2ecc71);
    height: 100%;
    width: 0%;
    border-radius: 10px;
    transition: width 0.5s ease;
    display: flex;
    align-items: center;
    justify-content: center;
    color: white;
    font-weight: 600;
    font-size: 14px;
}
.status {
    color: #2c3e50;
    font-weight: 600;
    margin-top: 15px;
    text-align: center;
    font-size: 18px;
}
.log-container {
    background: #2c3e50;
    color: #ecf0f1;
    border-radius: 8px;
    padding: 15px;
    margin-top: 20px;
    font-family: 'Courier New', monospace;
    font-size: 14px;
    max-height: 300px;
    overflow-y: auto;
    white-space: pre-wrap;
}
.log-entry {
    margin-bottom: 5px;
}
.log-info {
    color: #3498db;
}
.log-success {
    color: #2ecc71;
}
.log-error {
    color: #e74c3c;
}
.log-warning {
    color: #f39c12;
}
.button {
    background: linear-gradient(90deg, #3498db, #2ecc71);
    color: white;
    border: none;
    padding: 10px 20px;
    border-radius: 6px;
    cursor: pointer;
    font-weight: 600;
    transition: all 0.3s ease;
    margin: 5px;
}
.button:hover {
    transform: translateY(-2px);
    box-shadow: 0 4px 8px rgba(0,0,0,0.2);
}
.model-card {
    background: linear-gradient(135deg, #3498db, #2ecc71);
    color: white;
    border-radius: 12px;
    padding: 15px;
    margin-bottom: 15px;
    box-shadow: 0 4px 15px rgba(0,0,0,0.1);
}
.model-name {
    font-size: 18px;
    font-weight: 600;
    margin-bottom: 5px;
}
.model-desc {
    font-size: 14px;
    opacity: 0.9;
}
</style>
"""

# Global variables for logging
log_entries = []
log_lock = threading.Lock()

def add_log_entry(message, log_type="info"):
    timestamp = datetime.now().strftime("%H:%M:%S.%f")[:-3]
    with log_lock:
        log_entries.append(f"[{timestamp}] {message}")
        # Keep only the last 100 log entries
        if len(log_entries) > 100:
            log_entries.pop(0)

    # Update log display
    log_class = f"log-{log_type}"
    log_html = f'<div class="log-entry {log_class}">[{timestamp}] {message}</div>'
    display(HTML(log_html))

def update_progress(percent, status_text=""):
    progress_html = f"""
    <div class="progress-container">
        <div class="progress-bar" style="width: {percent}%">
            {percent}%
        </div>
    </div>
    <div class="status">{status_text}</div>
    """
    display(HTML(progress_html))

def show_status(message, status_type="info"):
    if status_type == "success":
        display(HTML(f'<div class="success">✓ {message}</div>'))
    elif status_type == "error":
        display(HTML(f'<div class="error">✗ {message}</div>'))
    elif status_type == "warning":
        display(HTML(f'<div class="warning">⚠ {message}</div>'))
    else:
        display(HTML(f'<div class="info">• {message}</div>'))

    add_log_entry(message, status_type)

# Function to execute command and show output
def execute_command(cmd, description):
    show_status(f"Executing: {' '.join(cmd)}", "info")
    add_log_entry(f"Executing command: {' '.join(cmd)}", "info")

    try:
        result = subprocess.run(cmd, capture_output=True, text=True)
        if result.returncode == 0:
            show_status(f"Successfully executed: {description}", "success")
            add_log_entry(f"Successfully executed: {description}", "success")
            return True, result.stdout
        else:
            show_status(f"Failed to execute: {description}", "error")
            add_log_entry(f"Failed to execute: {description}: {result.stderr}", "error")
            return False, result.stderr
    except Exception as e:
        show_status(f"Error executing {description}: {str(e)}", "error")
        add_log_entry(f"Error executing {description}: {str(e)}", "error")
        return False, str(e)

# Function to check if directory exists
def directory_exists(path):
    return os.path.exists(path) and os.path.isdir(path)

# Function to create symbolic link
def create_symlink(source, target):
    if os.path.exists(target):
        if os.path.islink(target):
            os.unlink(target)
        else:
            shutil.rmtree(target) if os.path.isdir(target) else os.remove(target)

    try:
        os.symlink(source, target)
        show_status(f"Created symlink: {target} -> {source}", "success")
        add_log_entry(f"Created symlink: {target} -> {source}", "success")
        return True
    except Exception as e:
        show_status(f"Failed to create symlink {target} -> {source}: {str(e)}", "error")
        add_log_entry(f"Failed to create symlink {target} -> {source}: {str(e)}", "error")
        return False

# Initialize
display(HTML(CSS))
display(HTML('<div class="container">'))
display(HTML('<div class="title">Clone and Configure ComfyUI</div>'))
display(HTML('<div class="card">'))

update_progress(0, "Initializing ComfyUI setup...")
add_log_entry("Starting ComfyUI setup process", "info")

# Track operations
successful_ops = []
failed_ops = []
total_steps = 5

# Step 1: Clone ComfyUI
update_progress(20, "Cloning ComfyUI repository...")
add_log_entry("Cloning ComfyUI repository", "info")
step = 1

if directory_exists('/content/ComfyUI'):
    show_status("ComfyUI directory already exists", "info")
    add_log_entry("ComfyUI directory already exists", "info")
    successful_ops.append("ComfyUI directory check")
else:
    success, output = execute_command(
        ['git', 'clone', 'https://github.com/comfyanonymous/ComfyUI', '/content/ComfyUI'],
        "Clone ComfyUI repository"
    )

    if success:
        successful_ops.append("Clone ComfyUI")
    else:
        failed_ops.append("Clone ComfyUI")

# Step 2: Change to ComfyUI directory
update_progress(40, "Changing to ComfyUI directory...")
add_log_entry("Changing to ComfyUI directory", "info")
step += 1

try:
    os.chdir('/content/ComfyUI')
    show_status("Changed to ComfyUI directory", "success")
    add_log_entry("Changed to ComfyUI directory", "success")
    successful_ops.append("Change directory")
except Exception as e:
    show_status(f"Failed to change directory: {str(e)}", "error")
    add_log_entry(f"Failed to change directory: {str(e)}", "error")
    failed_ops.append("Change directory")

# Step 3: Install ComfyUI essentials
update_progress(60, "Installing ComfyUI essentials...")
add_log_entry("Installing ComfyUI essentials", "info")
step += 1

packages = ['aiohttp', 'einops', 'torchsde', 'safetensors', 'kornia', 'spandrel']
for pkg in packages:
    success, output = execute_command(
        [sys.executable, '-m', 'pip', 'install', pkg, '--upgrade-strategy', 'only-if-needed'],
        f"Install {pkg}"
    )

    if success:
        successful_ops.append(f"Install {pkg}")
    else:
        failed_ops.append(f"Install {pkg}")

# Step 4: Create symbolic links
update_progress(80, "Creating symbolic links...")
add_log_entry("Creating symbolic links", "info")
step += 1

symlinks = [
    ('/content/ComfyUI_Models/checkpoints', '/content/ComfyUI/models/checkpoints'),
    ('/content/ComfyUI_Models/loras', '/content/ComfyUI/models/loras'),
    ('/content/ComfyUI_Models/controlnet', '/content/ComfyUI/models/controlnet'),
    ('/content/ComfyUI_Models/upscale_models', '/content/ComfyUI/models/upscale_models'),
    ('/content/ComfyUI_Models/vae', '/content/ComfyUI/models/vae'),
    ('/content/ComfyUI_Outputs', '/content/ComfyUI/output')
]

for source, target in symlinks:
    if create_symlink(source, target):
        successful_ops.append(f"Create symlink {target}")
    else:
        failed_ops.append(f"Create symlink {target}")

# Step 5: Verify directory structure
update_progress(100, "Verifying directory structure...")
add_log_entry("Verifying directory structure", "info")
step += 1

# Display directory structure
display(HTML('<div class="subtitle">ComfyUI Directory Structure</div>'))
display(HTML('<div class="info">/content/ComfyUI/</div>'))
display(HTML('<div class="info">├── models/</div>'))
display(HTML('<div class="info">&nbsp;&nbsp;├── checkpoints -> /content/ComfyUI_Models/checkpoints</div>'))
display(HTML('<div class="info">&nbsp;&nbsp;├── loras -> /content/ComfyUI_Models/loras</div>'))
display(HTML('<div class="info">&nbsp;&nbsp;├── controlnet -> /content/ComfyUI_Models/controlnet</div>'))
display(HTML('<div class="info">&nbsp;&nbsp;├── upscale_models -> /content/ComfyUI_Models/upscale_models</div>'))
display(HTML('<div class="info">&nbsp;&nbsp;└── vae -> /content/ComfyUI_Models/vae</div>'))
display(HTML('<div class="info">├── custom_nodes/</div>'))
display(HTML('<div class="info">└── output -> /content/ComfyUI_Outputs</div>'))

# Check if symlinks were created correctly
for _, target in symlinks:
    if os.path.islink(target) and os.path.exists(target):
        successful_ops.append(f"Verify symlink {target}")
    else:
        failed_ops.append(f"Verify symlink {target}")

# Final progress update
update_progress(100, "ComfyUI setup completed!")
add_log_entry("ComfyUI setup process completed", "success")

# Summary report
display(HTML('<div class="subtitle">ComfyUI Setup Summary</div>'))
display(HTML(f'<div class="info">• Total operations: {len(successful_ops) + len(failed_ops)}</div>'))
display(HTML(f'<div class="info">• Successful operations: {len(successful_ops)}</div>'))
display(HTML(f'<div class="info">• Failed operations: {len(failed_ops)}</div>'))

if failed_ops:
    display(HTML('<div class="error">Some operations failed</div>'))
    for op in failed_ops:
        display(HTML(f'<div class="error">- {op}</div>'))
else:
    display(HTML('<div class="success">All operations completed successfully</div>'))

display(HTML('</div>'))  # Close card
display(HTML('</div>'))  # Close container

# Print summary report
print("\n" + "="*50)
print("SUMMARY REPORT")
print("="*50)
print(f"Status: ComfyUI Setup")
print(f"Total Operations: {len(successful_ops) + len(failed_ops)}")
print(f"Successful: {len(successful_ops)}")
print(f"Failed: {len(failed_ops)}")

if successful_ops:
    print("\nSuccessful Operations:")
    for op in successful_ops:
        print(f"  - {op}")

if failed_ops:
    print("\nFailed Operations:")
    for op in failed_ops:
        print(f"  - {op}")

print(f"\nNext Step: Install Custom Nodes")
print("="*50)

In [ ]:
# ===== STEP 6: INSTALL CUSTOM NODES =====
from IPython.display import HTML, display, clear_output
import subprocess
import sys
import os
import time
import threading
from datetime import datetime

# Define CSS styles (since we can't reuse from previous cells)
CSS = """
<style>
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700&display=swap');
body {
    font-family: 'Inter', sans-serif;
    background: linear-gradient(135deg, #f5f7fa 0%, #e4e8f0 100%);
    margin: 0;
    padding: 0;
}
.container {
    max-width: 1000px;
    margin: 0 auto;
    padding: 20px;
}
.title {
    color: #2c3e50;
    font-size: 28px;
    font-weight: 700;
    margin-bottom: 20px;
    text-align: center;
    background: linear-gradient(90deg, #3498db, #2ecc71);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    text-shadow: 0px 2px 4px rgba(0,0,0,0.1);
}
.subtitle {
    color: #34495e;
    font-size: 20px;
    font-weight: 600;
    margin-top: 20px;
    margin-bottom: 15px;
    padding-bottom: 8px;
    border-bottom: 2px solid #3498db;
}
.card {
    background: white;
    border-radius: 12px;
    padding: 20px;
    margin-bottom: 20px;
    box-shadow: 0 4px 15px rgba(0,0,0,0.05);
    transition: transform 0.3s ease, box-shadow 0.3s ease;
}
.card:hover {
    transform: translateY(-5px);
    box-shadow: 0 8px 25px rgba(0,0,0,0.1);
}
.info {
    color: #34495e;
    margin-bottom: 12px;
    font-size: 16px;
    line-height: 1.5;
}
.success {
    color: #2ecc71;
    font-weight: 600;
}
.error {
    color: #e74c3c;
    font-weight: 600;
}
.warning {
    color: #f39c12;
    font-weight: 600;
}
.progress-container {
    background: #ecf0f1;
    border-radius: 10px;
    height: 24px;
    margin: 15px 0;
    overflow: hidden;
    position: relative;
}
.progress-bar {
    background: linear-gradient(90deg, #3498db, #2ecc71);
    height: 100%;
    width: 0%;
    border-radius: 10px;
    transition: width 0.5s ease;
    display: flex;
    align-items: center;
    justify-content: center;
    color: white;
    font-weight: 600;
    font-size: 14px;
}
.status {
    color: #2c3e50;
    font-weight: 600;
    margin-top: 15px;
    text-align: center;
    font-size: 18px;
}
</style>
"""

# Global variables for logging
log_entries = []
log_lock = threading.Lock()

def add_log_entry(message, log_type="info"):
    timestamp = datetime.now().strftime("%H:%M:%S.%f")[:-3]
    with log_lock:
        log_entries.append(f"[{timestamp}] {message}")
        # Keep only the last 100 log entries
        if len(log_entries) > 100:
            log_entries.pop(0)

    # Update log display
    log_class = f"log-{log_type}"
    log_html = f'<div class="log-entry {log_class}">[{timestamp}] {message}</div>'
    display(HTML(log_html))

def update_progress(percent, status_text=""):
    progress_html = f"""
    <div class="progress-container">
        <div class="progress-bar" style="width: {percent}%">
            {percent}%
        </div>
    </div>
    <div class="status">{status_text}</div>
    """
    display(HTML(progress_html))

def show_status(message, status_type="info"):
    if status_type == "success":
        display(HTML(f'<div class="success">✓ {message}</div>'))
    elif status_type == "error":
        display(HTML(f'<div class="error">✗ {message}</div>'))
    elif status_type == "warning":
        display(HTML(f'<div class="warning">⚠ {message}</div>'))
    else:
        display(HTML(f'<div class="info">• {message}</div>'))

    add_log_entry(message, status_type)

# Initialize
display(HTML(CSS))
display(HTML('<div class="container">'))
display(HTML('<div class="title">Install Custom Nodes</div>'))
display(HTML('<div class="card">'))
update_progress(0, "Initializing custom nodes installation...")
add_log_entry("Starting custom nodes installation process", "info")

# Function to clone repository
def clone_repository(url, target_dir, description):
    # Check if directory already exists
    if os.path.exists(target_dir):
        show_status(f"Directory already exists: {target_dir}", "info")
        add_log_entry(f"Directory already exists: {target_dir}", "info")
        return True

    show_status(f"Cloning: {description}", "info")
    add_log_entry(f"Cloning repository: {description}", "info")

    try:
        result = subprocess.run(['git', 'clone', url, target_dir],
                              capture_output=True, text=True)
        if result.returncode == 0:
            show_status(f"Successfully cloned: {description}", "success")
            add_log_entry(f"Successfully cloned: {description}", "success")
            return True
        else:
            show_status(f"Failed to clone {description}: {result.stderr}", "error")
            add_log_entry(f"Failed to clone {description}: {result.stderr}", "error")
            return False
    except Exception as e:
        show_status(f"Error cloning {description}: {str(e)}", "error")
        add_log_entry(f"Error cloning {description}: {str(e)}", "error")
        return False

# Function to install node group
def install_node_group(category_name, nodes):
    display(HTML(f'<div class="subtitle">{category_name}</div>'))
    add_log_entry(f"Installing node group: {category_name}", "info")

    installed = []
    skipped = []
    failed = []

    for node in nodes:
        success = clone_repository(node['url'], node['target'], node['description'])

        if success:
            if os.path.exists(node['target']):
                skipped.append(node['description'])
            else:
                installed.append(node['description'])
        else:
            failed.append(node['description'])

        time.sleep(0.3)  # Small delay for visual feedback

    return installed, skipped, failed

# Change to custom_nodes directory
try:
    os.chdir('/content/ComfyUI/custom_nodes')
    show_status("Changed to custom_nodes directory", "success")
    add_log_entry("Changed to custom_nodes directory", "success")
except Exception as e:
    show_status(f"Failed to change directory: {str(e)}", "error")
    add_log_entry(f"Failed to change directory: {str(e)}", "error")

# Define node groups
node_groups = [
    {
        "name": "Essential Nodes",
        "nodes": [
            {
                "url": "https://github.com/ltdrdata/ComfyUI-Manager",
                "target": "/content/ComfyUI/custom_nodes/ComfyUI-Manager",
                "description": "ComfyUI Manager"
            },
            {
                "url": "https://github.com/Fannovel16/comfyui_controlnet_aux",
                "target": "/content/ComfyUI/custom_nodes/comfyui_controlnet_aux",
                "description": "ControlNet Auxiliary"
            }
        ]
    },
    {
        "name": "Core Utilities",
        "nodes": [
            {
                "url": "https://github.com/WASasquatch/was-node-suite-comfyui",
                "target": "/content/ComfyUI/custom_nodes/was-node-suite-comfyui",
                "description": "WAS Node Suite"
            }
        ]
    },
    {
        "name": "Advanced Sampling",
        "nodes": [
            {
                "url": "https://github.com/BlenderNeko/ComfyUI_ADV_CLIP_emb",
                "target": "/content/ComfyUI/custom_nodes/ComfyUI_ADV_CLIP_emb",
                "description": "Advanced CLIP Embedding"
            }
        ]
    },
    {
        "name": "Face Enhancement",
        "nodes": [
            {
                "url": "https://github.com/teaser003838/ComfyUI-ReActor-mine-v3",
                "target": "/content/ComfyUI/custom_nodes/comfyui-reactor-node",
                "description": "Reactor Face Enhancement"
            }
        ]
    },
    {
        "name": "Detection and Segmentation",
        "nodes": [
            {
                "url": "https://github.com/ltdrdata/ComfyUI-Impact-Pack",
                "target": "/content/ComfyUI/custom_nodes/ComfyUI-Impact-Pack",
                "description": "Impact Pack"
            },
            {
                "url": "https://github.com/ltdrdata/ComfyUI-Impact-Subpack",
                "target": "/content/ComfyUI/custom_nodes/ComfyUI-Impact-Subpack",
                "description": "Impact Subpack"
            }
        ]
    },
    {
        "name": "Upscaling",
        "nodes": [
            {
                "url": "https://github.com/ssitu/ComfyUI_UltimateSDUpscale",
                "target": "/content/ComfyUI/custom_nodes/ComfyUI_UltimateSDUpscale",
                "description": "Ultimate SD Upscale"
            },
            {
                "url": "https://github.com/BlenderNeko/ComfyUI_TiledKSampler",
                "target": "/content/ComfyUI/custom_nodes/ComfyUI_TiledKSampler",
                "description": "Tiled K Sampler"
            }
        ]
    },
    {
        "name": "Additional Features",
        "nodes": [
            {
                "url": "https://github.com/cubiq/ComfyUI_IPAdapter_plus",
                "target": "/content/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus",
                "description": "IPAdapter Plus"
            },
            {
                "url": "https://github.com/pythongosssss/ComfyUI-WD14-Tagger",
                "target": "/content/ComfyUI/custom_nodes/ComfyUI-WD14-Tagger",
                "description": "WD14 Tagger"
            }
        ]
    }
]

# Track installation results
all_installed = []
all_skipped = []
all_failed = []
total_groups = len(node_groups)

# Install node groups
for i, group in enumerate(node_groups):
    progress = int((i / total_groups) * 100)
    update_progress(progress, f"Installing {group['name']} nodes...")
    add_log_entry(f"Installing node group: {group['name']}", "info")

    installed, skipped, failed = install_node_group(group['name'], group['nodes'])
    all_installed.extend(installed)
    all_skipped.extend(skipped)
    all_failed.extend(failed)

    time.sleep(0.5)  # Small delay for visual feedback

# Final progress update
update_progress(100, "Custom nodes installation completed!")
add_log_entry("Custom nodes installation process completed", "success")

# Display installed nodes list
display(HTML('<div class="subtitle">Installed Custom Nodes</div>'))
for node in all_installed:
    display(HTML(f'<div class="success">✓ {node}</div>'))

# Summary report
display(HTML('<div class="subtitle">Custom Nodes Installation Summary</div>'))
display(HTML(f'<div class="info">• Total node groups processed: {total_groups}</div>'))
display(HTML(f'<div class="info">• Successfully installed nodes: {len(all_installed)}</div>'))
display(HTML(f'<div class="info">• Skipped (already installed): {len(all_skipped)}</div>'))
display(HTML(f'<div class="info">• Failed installations: {len(all_failed)}</div>'))

if all_failed:
    display(HTML('<div class="error">Some nodes failed to install</div>'))
    for node in all_failed:
        display(HTML(f'<div class="error">- {node}</div>'))
else:
    display(HTML('<div class="success">All custom nodes installed successfully</div>'))

display(HTML('</div>'))  # Close card
display(HTML('</div>'))  # Close container

# Print summary report
print("\n" + "="*50)
print("SUMMARY REPORT")
print("="*50)
print(f"Status: Custom Nodes Installation")
print(f"Total Node Groups: {total_groups}")
print(f"Successfully Installed: {len(all_installed)}")
print(f"Skipped: {len(all_skipped)}")
print(f"Failed: {len(all_failed)}")

if all_installed:
    print("\nInstalled Nodes:")
    for node in all_installed:
        print(f"  - {node}")

if all_skipped:
    print("\nSkipped Nodes:")
    for node in all_skipped:
        print(f"  - {node}")

if all_failed:
    print("\nFailed Nodes:")
    for node in all_failed:
        print(f"  - {node}")

print(f"\nNext Step: Install Node Dependencies")
print("="*50)

In [ ]:
# ===== COMPREHENSIVE VERIFICATION: STEPS 1-6 =====
from IPython.display import HTML, display, clear_output
import subprocess
import sys
import os
import time
import importlib
import threading
from datetime import datetime

# Define colored font styles
CSS = """
<style>
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700&display=swap');
body {
    font-family: 'Inter', sans-serif;
    background: linear-gradient(135deg, #f5f7fa 0%, #e4e8f0 100%);
    margin: 0;
    padding: 0;
}
.container {
    max-width: 1200px;
    margin: 0 auto;
    padding: 20px;
}
.title {
    color: #2c3e50;
    font-size: 32px;
    font-weight: 700;
    margin-bottom: 25px;
    text-align: center;
    background: linear-gradient(90deg, #3498db, #2ecc71, #e74c3c);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    text-shadow: 0px 2px 4px rgba(0,0,0,0.1);
}
.subtitle {
    color: #34495e;
    font-size: 22px;
    font-weight: 600;
    margin-top: 25px;
    margin-bottom: 15px;
    padding-bottom: 8px;
    border-bottom: 3px solid #3498db;
}
.card {
    background: white;
    border-radius: 15px;
    padding: 25px;
    margin-bottom: 25px;
    box-shadow: 0 8px 25px rgba(0,0,0,0.08);
    transition: transform 0.3s ease, box-shadow 0.3s ease;
}
.card:hover {
    transform: translateY(-5px);
    box-shadow: 0 12px 35px rgba(0,0,0,0.12);
}
.info {
    color: #34495e;
    margin-bottom: 15px;
    font-size: 16px;
    line-height: 1.6;
}
.success {
    color: #27ae60;
    font-weight: 600;
    font-size: 16px;
}
.error {
    color: #e74c3c;
    font-weight: 600;
    font-size: 16px;
}
.warning {
    color: #f39c12;
    font-weight: 600;
    font-size: 16px;
}
.highlight {
    color: #3498db;
    font-weight: 600;
}
.progress-container {
    background: #ecf0f1;
    border-radius: 12px;
    height: 28px;
    margin: 20px 0;
    overflow: hidden;
    position: relative;
}
.progress-bar {
    background: linear-gradient(90deg, #3498db, #2ecc71);
    height: 100%;
    width: 0%;
    border-radius: 12px;
    transition: width 0.5s ease;
    display: flex;
    align-items: center;
    justify-content: center;
    color: white;
    font-weight: 600;
    font-size: 16px;
}
.status {
    color: #2c3e50;
    font-weight: 600;
    margin-top: 20px;
    text-align: center;
    font-size: 20px;
}
.package-table {
    width: 100%;
    border-collapse: collapse;
    margin: 20px 0;
    background: white;
    border-radius: 10px;
    overflow: hidden;
    box-shadow: 0 4px 15px rgba(0,0,0,0.05);
}
.package-table th {
    background: linear-gradient(90deg, #3498db, #2ecc71);
    color: white;
    padding: 15px;
    text-align: left;
    font-weight: 600;
    font-size: 16px;
}
.package-table td {
    padding: 12px 15px;
    border-bottom: 1px solid #ecf0f1;
    font-size: 15px;
}
.package-table tr:last-child td {
    border-bottom: none;
}
.package-table tr:nth-child(even) {
    background-color: #f8f9fa;
}
.status-badge {
    padding: 6px 12px;
    border-radius: 20px;
    font-size: 13px;
    font-weight: 600;
    text-transform: uppercase;
}
.status-success {
    background-color: #d4edda;
    color: #155724;
}
.status-warning {
    background-color: #fff3cd;
    color: #856404;
}
.status-error {
    background-color: #f8d7da;
    color: #721c24;
}
.summary-card {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    border-radius: 15px;
    padding: 25px;
    margin: 20px 0;
    box-shadow: 0 8px 25px rgba(0,0,0,0.15);
}
.summary-stat {
    font-size: 24px;
    font-weight: 700;
    margin-bottom: 5px;
}
.summary-label {
    font-size: 14px;
    opacity: 0.9;
    text-transform: uppercase;
    letter-spacing: 1px;
}
</style>
"""

# Global variables for logging
log_entries = []
log_lock = threading.Lock()

def add_log_entry(message, log_type="info"):
    timestamp = datetime.now().strftime("%H:%M:%S.%f")[:-3]
    with log_lock:
        log_entries.append(f"[{timestamp}] {message}")
        if len(log_entries) > 100:
            log_entries.pop(0)
    log_class = f"log-{log_type}"
    log_html = f'<div class="log-entry {log_class}">[{timestamp}] {message}</div>'
    display(HTML(log_html))

def update_progress(percent, status_text=""):
    progress_html = f"""
    <div class="progress-container">
        <div class="progress-bar" style="width: {percent}%">
            {percent}%
        </div>
    </div>
    <div class="status">{status_text}</div>
    """
    display(HTML(progress_html))

def show_status(message, status_type="info"):
    if status_type == "success":
        display(HTML(f'<div class="success">✓ {message}</div>'))
    elif status_type == "error":
        display(HTML(f'<div class="error">✗ {message}</div>'))
    elif status_type == "warning":
        display(HTML(f'<div class="warning">⚠ {message}</div>'))
    else:
        display(HTML(f'<div class="info">• {message}</div>'))
    add_log_entry(message, status_type)

def get_package_version(package_name):
    """Get the installed version of a package"""
    try:
        result = subprocess.run([sys.executable, '-m', 'pip', 'show', package_name],
                              capture_output=True, text=True)
        if result.returncode == 0:
            for line in result.stdout.split('\n'):
                if line.startswith('Version:'):
                    return line.split(':')[1].strip()
        return None
    except Exception:
        return None

def check_package_status(package_name, expected_version=None):
    """Check if a package is installed and verify its version"""
    try:
        # Try to import the package
        if package_name == 'opencv-python':
            import cv2
            actual_version = cv2.__version__
        elif package_name == 'pillow':
            import PIL
            actual_version = PIL.__version__
        elif package_name == 'torch':
            import torch
            actual_version = torch.__version__.split('+')[0]  # Remove CUDA suffix
        elif package_name == 'torchvision':
            import torchvision
            actual_version = torchvision.__version__.split('+')[0]  # Remove CUDA suffix
        elif package_name == 'torchaudio':
            import torchaudio
            actual_version = torchaudio.__version__.split('+')[0]  # Remove CUDA suffix
        else:
            module = __import__(package_name)
            actual_version = getattr(module, '__version__', 'Unknown')

        # Check if version matches expected
        if expected_version:
            if actual_version == expected_version:
                return "success", actual_version, f"Correct version installed"
            else:
                return "warning", actual_version, f"Expected {expected_version}, found {actual_version}"
        else:
            return "success", actual_version, "Package installed"

    except ImportError:
        return "error", None, "Package not installed"
    except Exception as e:
        return "error", None, f"Error checking package: {str(e)}"

def check_directory_exists(path, description):
    """Check if a directory exists"""
    if os.path.exists(path) and os.path.isdir(path):
        return "success", f"{description} exists"
    else:
        return "error", f"{description} missing"

def check_symlink_exists(target, description):
    """Check if a symbolic link exists and is valid"""
    if os.path.islink(target) and os.path.exists(target):
        return "success", f"{description} symlink valid"
    else:
        return "error", f"{description} symlink missing or broken"

# Initialize
display(HTML(CSS))
display(HTML('<div class="container">'))
display(HTML('<div class="title">🔍 Comprehensive Verification: Steps 1-6</div>'))
display(HTML('<div class="card">'))

update_progress(0, "Initializing comprehensive verification...")
add_log_entry("Starting comprehensive verification process", "info")

# Track verification results
all_results = []
step_progress = 0

# Step 1: Check Directory Structure
update_progress(10, "Verifying Step 1: Directory Structure...")
add_log_entry("Verifying Step 1: Directory Structure", "info")

display(HTML('<div class="subtitle">Step 1: Directory Structure Verification</div>'))

required_dirs = [
    ('/content/ComfyUI_Models', 'ComfyUI_Models directory'),
    ('/content/ComfyUI_Outputs', 'ComfyUI_Outputs directory'),
    ('/content/ComfyUI_Models/checkpoints', 'Checkpoints directory'),
    ('/content/ComfyUI_Models/loras', 'LoRAs directory'),
    ('/content/ComfyUI_Models/loras/face_enhancement', 'Face enhancement LoRA directory'),
    ('/content/ComfyUI_Models/loras/lighting_photography', 'Lighting photography LoRA directory'),
    ('/content/ComfyUI_Models/loras/style_effects', 'Style effects LoRA directory'),
    ('/content/ComfyUI_Models/loras/detail_boost', 'Detail boost LoRA directory'),
    ('/content/ComfyUI_Models/controlnet', 'ControlNet directory'),
    ('/content/ComfyUI_Models/upscale_models', 'Upscale models directory'),
    ('/content/ComfyUI_Models/vae', 'VAE directory')
]

step1_success = 0
step1_total = len(required_dirs)

for dir_path, description in required_dirs:
    status, message = check_directory_exists(dir_path, description)
    all_results.append({
        'step': 'Step 1',
        'category': 'Directory Structure',
        'item': description,
        'status': status,
        'expected': 'Exists',
        'actual': message
    })

    if status == "success":
        step1_success += 1
        show_status(f"✓ {description}", "success")
    else:
        show_status(f"✗ {description}", "error")

step_progress = 15

# Step 2: Check Foundation Packages
update_progress(step_progress, "Verifying Step 2-3: Foundation Packages...")
add_log_entry("Verifying Step 2-3: Foundation Packages", "info")

display(HTML('<div class="subtitle">Step 2-3: Foundation Packages Verification</div>'))

foundation_packages = [
    ('numpy', '1.26.2'),
    ('torch', '2.1.2'),
    ('torchvision', '0.16.2'),
    ('torchaudio', '2.1.2'),
    ('xformers', '0.0.23')
]

step2_success = 0
step2_total = len(foundation_packages)

for package_name, expected_version in foundation_packages:
    status, actual_version, message = check_package_status(package_name, expected_version)
    all_results.append({
        'step': 'Step 2-3',
        'category': 'Foundation Packages',
        'item': package_name,
        'status': status,
        'expected': expected_version,
        'actual': actual_version or 'Not installed'
    })

    if status == "success":
        step2_success += 1
        show_status(f"✓ {package_name}: {actual_version}", "success")
    elif status == "warning":
        show_status(f"⚠ {package_name}: Expected {expected_version}, found {actual_version}", "warning")
    else:
        show_status(f"✗ {package_name}: {message}", "error")

step_progress = 40

# Step 3: Check Dependencies
update_progress(step_progress, "Verifying Step 4: Dependencies...")
add_log_entry("Verifying Step 4: Dependencies", "info")

display(HTML('<div class="subtitle">Step 4: Dependencies Verification</div>'))

dependency_packages = [
    ('scipy', '1.13.0'),
    ('scikit-image', '0.22.0'),
    ('matplotlib', '3.8.2'),
    ('opencv-python', '4.8.0.76'),
    ('opencv-contrib-python', '4.8.0.76'),
    ('pillow', '10.2.0'),
    ('transformers', '4.36.2'),
    ('diffusers', '0.25.0'),
    ('accelerate', '0.25.0'),
    ('safetensors', '0.4.1'),
    ('albumentations', '1.3.1'),
    ('ultralytics', '8.0.232'),
    ('insightface', '0.7.3'),
    ('gradio', '4.8.0')
]

step3_success = 0
step3_total = len(dependency_packages)

for package_name, expected_version in dependency_packages:
    status, actual_version, message = check_package_status(package_name, expected_version)
    all_results.append({
        'step': 'Step 4',
        'category': 'Dependencies',
        'item': package_name,
        'status': status,
        'expected': expected_version,
        'actual': actual_version or 'Not installed'
    })

    if status == "success":
        step3_success += 1
        show_status(f"✓ {package_name}: {actual_version}", "success")
    elif status == "warning":
        show_status(f"⚠ {package_name}: Expected {expected_version}, found {actual_version}", "warning")
    else:
        show_status(f"✗ {package_name}: {message}", "error")

step_progress = 70

# Step 4: Check ComfyUI Installation
update_progress(step_progress, "Verifying Step 5: ComfyUI Installation...")
add_log_entry("Verifying Step 5: ComfyUI Installation", "info")

display(HTML('<div class="subtitle">Step 5: ComfyUI Installation Verification</div>'))

# Check ComfyUI directory
comfyui_status, comfyui_message = check_directory_exists('/content/ComfyUI', 'ComfyUI directory')
all_results.append({
    'step': 'Step 5',
    'category': 'ComfyUI Installation',
    'item': 'ComfyUI directory',
    'status': comfyui_status,
    'expected': 'Exists',
    'actual': comfyui_message
})

if comfyui_status == "success":
    show_status(f"✓ ComfyUI directory exists", "success")
else:
    show_status(f"✗ ComfyUI directory missing", "error")

# Check symbolic links
symlinks = [
    ('/content/ComfyUI/models/checkpoints', 'Checkpoints symlink'),
    ('/content/ComfyUI/models/loras', 'LoRAs symlink'),
    ('/content/ComfyUI/models/controlnet', 'ControlNet symlink'),
    ('/content/ComfyUI/models/upscale_models', 'Upscale models symlink'),
    ('/content/ComfyUI/models/vae', 'VAE symlink'),
    ('/content/ComfyUI/output', 'Output symlink')
]

step4_success = 0
step4_total = len(symlinks) + 1  # +1 for ComfyUI directory

if comfyui_status == "success":
    step4_success += 1

for target, description in symlinks:
    status, message = check_symlink_exists(target, description)
    all_results.append({
        'step': 'Step 5',
        'category': 'ComfyUI Installation',
        'item': description,
        'status': status,
        'expected': 'Valid symlink',
        'actual': message
    })

    if status == "success":
        step4_success += 1
        show_status(f"✓ {description}", "success")
    else:
        show_status(f"✗ {description}", "error")

step_progress = 90

# Step 5: Check ComfyUI Essentials
update_progress(step_progress, "Verifying Step 5: ComfyUI Essentials...")
add_log_entry("Verifying Step 5: ComfyUI Essentials", "info")

display(HTML('<div class="subtitle">Step 5: ComfyUI Essentials Verification</div>'))

essential_packages = ['aiohttp', 'einops', 'torchsde', 'safetensors', 'kornia', 'spandrel']
step5_success = 0
step5_total = len(essential_packages)

for package in essential_packages:
    status, actual_version, message = check_package_status(package)
    all_results.append({
        'step': 'Step 5',
        'category': 'ComfyUI Essentials',
        'item': package,
        'status': status,
        'expected': 'Installed',
        'actual': actual_version or 'Not installed'
    })

    if status == "success":
        step5_success += 1
        show_status(f"✓ {package}: {actual_version}", "success")
    else:
        show_status(f"✗ {package}: {message}", "error")

# Final progress update
update_progress(100, "Comprehensive verification completed!")
add_log_entry("Comprehensive verification process completed", "success")

# Calculate overall statistics
total_items = len(all_results)
success_count = len([r for r in all_results if r['status'] == 'success'])
warning_count = len([r for r in all_results if r['status'] == 'warning'])
error_count = len([r for r in all_results if r['status'] == 'error'])
success_rate = (success_count / total_items) * 100

# Display summary
display(HTML('<div class="subtitle">📊 Overall Verification Summary</div>'))

summary_html = f"""
<div style="display: flex; justify-content: space-around; flex-wrap: wrap; margin: 20px 0;">
    <div class="summary-card" style="flex: 1; min-width: 200px; margin: 10px;">
        <div class="summary-stat">{total_items}</div>
        <div class="summary-label">Total Items</div>
    </div>
    <div class="summary-card" style="flex: 1; min-width: 200px; margin: 10px;">
        <div class="summary-stat">{success_count}</div>
        <div class="summary-label">Successful</div>
    </div>
    <div class="summary-card" style="flex: 1; min-width: 200px; margin: 10px;">
        <div class="summary-stat">{warning_count}</div>
        <div class="summary-label">Warnings</div>
    </div>
    <div class="summary-card" style="flex: 1; min-width: 200px; margin: 10px;">
        <div class="summary-stat">{error_count}</div>
        <div class="summary-label">Errors</div>
    </div>
    <div class="summary-card" style="flex: 1; min-width: 200px; margin: 10px;">
        <div class="summary-stat">{success_rate:.1f}%</div>
        <div class="summary-label">Success Rate</div>
    </div>
</div>
"""
display(HTML(summary_html))

# Display detailed results table
display(HTML('<div class="subtitle">📋 Detailed Verification Results</div>'))

table_html = """
<table class="package-table">
    <thead>
        <tr>
            <th>Step</th>
            <th>Category</th>
            <th>Item</th>
            <th>Status</th>
            <th>Expected</th>
            <th>Actual</th>
        </tr>
    </thead>
    <tbody>
"""

for result in all_results:
    status_class = f"status-{result['status']}"
    status_text = result['status'].title()

    table_html += f"""
        <tr>
            <td>{result['step']}</td>
            <td>{result['category']}</td>
            <td><strong>{result['item']}</strong></td>
            <td><span class="status-badge {status_class}">{status_text}</span></td>
            <td>{result['expected']}</td>
            <td>{result['actual']}</td>
        </tr>
    """

table_html += """
    </tbody>
</table>
"""
display(HTML(table_html))

# Recommendations
display(HTML('<div class="subtitle">💡 Recommendations</div>'))

if error_count > 0:
    show_status("⚠️ Critical issues found that may prevent ComfyUI from working properly", "error")
    show_status("🔧 Please fix the errors before proceeding", "warning")
elif warning_count > 0:
    show_status("⚠️ Version mismatches detected. ComfyUI may work but could have issues", "warning")
    show_status("🔧 Consider reinstalling packages with correct versions", "info")
else:
    show_status("✅ All packages and configurations are correct!", "success")
    show_status("🚀 Ready to proceed with the next steps", "success")

# Print summary report
print("\n" + "="*80)
print("COMPREHENSIVE VERIFICATION REPORT")
print("="*80)
print(f"Total Items Checked: {total_items}")
print(f"Successful: {success_count}")
print(f"Warnings: {warning_count}")
print(f"Errors: {error_count}")
print(f"Success Rate: {success_rate:.1f}%")
print("\nStep-by-Step Summary:")
print(f"Step 1 (Directory Structure): {step1_success}/{step1_total} successful")
print(f"Step 2-3 (Foundation Packages): {step2_success}/{step2_total} successful")
print(f"Step 4 (Dependencies): {step3_success}/{step3_total} successful")
print(f"Step 5 (ComfyUI Installation): {step4_success}/{step4_total} successful")
print(f"Step 5 (ComfyUI Essentials): {step5_success}/{step5_total} successful")

if error_count > 0:
    print("\n🚨 CRITICAL ISSUES FOUND:")
    error_items = [r for r in all_results if r['status'] == 'error']
    for item in error_items:
        print(f"  - {item['step']} - {item['item']}: {item['actual']}")

if warning_count > 0:
    print("\n⚠️ VERSION MISMATCHES:")
    warning_items = [r for r in all_results if r['status'] == 'warning']
    for item in warning_items:
        print(f"  - {item['item']}: Expected {item['expected']}, found {item['actual']}")

if error_count == 0 and warning_count == 0:
    print("\n✅ ALL CHECKS PASSED!")
    print("🎉 Your ComfyUI installation is ready!")
else:
    print("\n🔧 RECOMMENDATIONS:")
    if error_count > 0:
        print("  - Fix all critical errors before proceeding")
        print("  - Rerun the failed installation steps")
    if warning_count > 0:
        print("  - Consider reinstalling packages with correct versions")
        print("  - Use the Common Error Fixes cell for troubleshooting")

print("\n" + "="*80)

display(HTML('</div>'))  # Close card
display(HTML('</div>'))  # Close container

In [ ]:
# ===== STRICT VERSION PINNING PACKAGE FIX =====
from IPython.display import HTML, display, clear_output
import subprocess
import sys
import os
import time
import threading
from datetime import datetime

# Enhanced UI Styles
CSS = """
<style>
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700&display=swap');
body {
    font-family: 'Inter', sans-serif;
    background: linear-gradient(135deg, #f5f7fa 0%, #e4e8f0 100%);
    margin: 0;
    padding: 0;
}
.container {
    max-width: 1200px;
    margin: 0 auto;
    padding: 20px;
}
.title {
    color: #2c3e50;
    font-size: 32px;
    font-weight: 700;
    margin-bottom: 25px;
    text-align: center;
    background: linear-gradient(90deg, #e74c3c, #f39c12, #3498db);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
}
.subtitle {
    color: #34495e;
    font-size: 22px;
    font-weight: 600;
    margin-top: 25px;
    margin-bottom: 15px;
    padding-bottom: 8px;
    border-bottom: 3px solid #e74c3c;
}
.card {
    background: white;
    border-radius: 15px;
    padding: 25px;
    margin-bottom: 25px;
    box-shadow: 0 8px 25px rgba(0,0,0,0.08);
}
.info {
    color: #34495e;
    margin-bottom: 12px;
    font-size: 16px;
    line-height: 1.6;
}
.success {
    color: #27ae60;
    font-weight: 600;
    font-size: 16px;
}
.error {
    color: #e74c3c;
    font-weight: 600;
    font-size: 16px;
}
.warning {
    color: #f39c12;
    font-weight: 600;
    font-size: 16px;
}
.progress-container {
    background: #ecf0f1;
    border-radius: 12px;
    height: 30px;
    margin: 20px 0;
    overflow: hidden;
    position: relative;
}
.progress-bar {
    background: linear-gradient(90deg, #e74c3c, #f39c12);
    height: 100%;
    width: 0%;
    border-radius: 12px;
    transition: width 0.5s ease;
    display: flex;
    align-items: center;
    justify-content: center;
    color: white;
    font-weight: 600;
    font-size: 16px;
}
.status {
    color: #2c3e50;
    font-weight: 600;
    margin-top: 15px;
    text-align: center;
    font-size: 20px;
}
.package-table {
    width: 100%;
    border-collapse: collapse;
    margin: 20px 0;
    background: white;
    border-radius: 10px;
    overflow: hidden;
    box-shadow: 0 4px 15px rgba(0,0,0,0.05);
}
.package-table th {
    background: linear-gradient(90deg, #e74c3c, #f39c12);
    color: white;
    padding: 15px;
    text-align: left;
    font-weight: 600;
    font-size: 16px;
}
.package-table td {
    padding: 12px 15px;
    border-bottom: 1px solid #ecf0f1;
    font-size: 15px;
}
.package-table tr:last-child td {
    border-bottom: none;
}
.status-badge {
    padding: 6px 12px;
    border-radius: 20px;
    font-size: 13px;
    font-weight: 600;
    text-transform: uppercase;
}
.status-success {
    background-color: #d4edda;
    color: #155724;
}
.status-error {
    background-color: #f8d7da;
    color: #721c24;
}
.status-warning {
    background-color: #fff3cd;
    color: #856404;
}
.status-pending {
    background-color: #d1ecf1;
    color: #0c5460;
}
.status-running {
    background-color: #e2e3e5;
    color: #383d41;
}
.spinner {
    display: inline-block;
    width: 16px;
    height: 16px;
    border: 2px solid #f3f3f3;
    border-top: 2px solid #e74c3c;
    border-radius: 50%;
    animation: spin 1s linear infinite;
    margin-right: 8px;
}
@keyframes spin {
    0% { transform: rotate(0deg); }
    100% { transform: rotate(360deg); }
}
</style>
"""

# Initialize
display(HTML(CSS))
display(HTML('<div class="container">'))
display(HTML('<div class="title">🔒 STRICT VERSION PINNING PACKAGE FIX</div>'))
display(HTML('<div class="card">'))

# Global variables for tracking
package_results = []
current_operation = 0
total_operations = 0
progress_lock = threading.Lock()

def update_progress(percent, status_text=""):
    progress_html = f"""
    <div class="progress-container">
        <div class="progress-bar" style="width: {percent}%">
            {percent}%
        </div>
    </div>
    <div class="status">{status_text}</div>
    """
    display(HTML(progress_html))

def show_status(message, status_type="info"):
    if status_type == "success":
        display(HTML(f'<div class="success">✓ {message}</div>'))
    elif status_type == "error":
        display(HTML(f'<div class="error">✗ {message}</div>'))
    elif status_type == "warning":
        display(HTML(f'<div class="warning">⚠ {message}</div>'))
    else:
        display(HTML(f'<div class="info">• {message}</div>'))

def update_package_table():
    table_html = """
    <table class="package-table">
        <thead>
            <tr>
                <th>Package</th>
                <th>Status</th>
                <th>Details</th>
            </tr>
        </thead>
        <tbody>
    """
    for result in package_results:
        status_class = f"status-{result['status']}"
        status_text = result['status'].title()
        if result['status'] == 'running':
            status_text = '<span class="spinner"></span>Installing'

        table_html += f"""
            <tr>
                <td><strong>{result['package']}</strong></td>
                <td><span class="status-badge {status_class}">{status_text}</span></td>
                <td>{result['details']}</td>
            </tr>
        """
    table_html += """
        </tbody>
    </table>
    """
    display(HTML(table_html))

def run_command_with_progress(cmd, description, timeout=600):
    global current_operation, package_results

    # Update status to running
    with progress_lock:
        for result in package_results:
            if result['package'] == description:
                result['status'] = 'running'
                result['details'] = 'Installation in progress...'
                break
        update_package_table()

    show_status(f"🔄 Processing: {description}", "info")

    try:
        # Start a timer thread for progress updates
        def progress_updater():
            elapsed = 0
            while elapsed < timeout:
                time.sleep(5)
                elapsed += 5
                with progress_lock:
                    for result in package_results:
                        if result['package'] == description and result['status'] == 'running':
                            result['details'] = f'Installing... ({elapsed}s elapsed)'
                            update_package_table()
                if elapsed >= timeout:
                    break

        progress_thread = threading.Thread(target=progress_updater)
        progress_thread.daemon = True
        progress_thread.start()

        # Run the command
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=timeout)

        # Update the result
        with progress_lock:
            for res in package_results:
                if res['package'] == description:
                    if result.returncode == 0:
                        res['status'] = 'success'
                        res['details'] = 'Successfully installed'
                        show_status(f"✅ Success: {description}", "success")
                    else:
                        res['status'] = 'error'
                        res['details'] = f"Error: {result.stderr[:200]}..."
                        show_status(f"❌ Failed: {description}", "error")
                    break
            update_package_table()

        return result.returncode == 0

    except subprocess.TimeoutExpired:
        with progress_lock:
            for res in package_results:
                if res['package'] == description:
                    res['status'] = 'warning'
                    res['details'] = f'Timeout after {timeout}s'
                    break
            update_package_table()
        show_status(f"⏱️ Timeout: {description}", "warning")
        return False
    except Exception as e:
        with progress_lock:
            for res in package_results:
                if res['package'] == description:
                    res['status'] = 'error'
                    res['details'] = f"Exception: {str(e)}"
                    break
            update_package_table()
        show_status(f"💥 Exception: {description} - {str(e)}", "error")
        return False

def initialize_package_list(packages, operation_type):
    global package_results
    for package in packages:
        package_results.append({
            'package': f"{operation_type}: {package}",
            'status': 'pending',
            'details': 'Waiting to process...'
        })

# First, create the constraints file with all required versions
constraints_content = """
numpy==1.26.2
torch==2.1.2+cu121
torchvision==0.16.2+cu121
torchaudio==2.1.2+cu121
xformers==0.0.23
scipy==1.13.0
scikit-image==0.22.0
matplotlib==3.8.2
opencv-python==4.8.0.76
opencv-contrib-python==4.8.0.76
pillow==10.2.0
transformers==4.36.2
diffusers==0.25.0
accelerate==0.25.0
safetensors==0.4.1
albumentations==1.3.1
ultralytics==8.0.232
insightface==0.7.3
gradio==4.8.0
Cython
onnx>=1.15.0
"""

constraints_file = '/content/pip_constraints.txt'

try:
    with open(constraints_file, 'w') as f:
        f.write(constraints_content)
    show_status("✅ Created pip constraints file", "success")
except Exception as e:
    show_status(f"❌ Failed to create constraints file: {e}", "error")

# Set environment variables to prevent upgrades
os.environ['PIP_CONSTRAINT'] = constraints_file
os.environ['PIP_NO_UPGRADE'] = '1'
os.environ['PIP_UPGRADE_STRATEGY'] = 'only-if-needed'

# Calculate total operations
problematic_packages = [
    'torch', 'torchvision', 'torchaudio', 'numpy', 'scipy',
    'matplotlib', 'opencv-python', 'opencv-contrib-python', 'pillow',
    'safetensors', 'albumentations', 'scikit-image', 'diffusers',
    'insightface', 'xformers'
]

foundation_packages = [
    "numpy==1.26.2",
    "torch==2.1.2+cu121",
    "torchvision==0.16.2+cu121",
    "torchaudio==2.1.2+cu121",
    "xformers==0.0.23"
]

dependency_packages = [
    "scipy==1.13.0",
    "scikit-image==0.22.0",
    "matplotlib==3.8.2",
    "opencv-python==4.8.0.76",
    "opencv-contrib-python==4.8.0.76",
    "pillow==10.2.0",
    "transformers==4.36.2",
    "diffusers==0.25.0",
    "accelerate==0.25.0",
    "safetensors==0.4.1",
    "albumentations==1.3.1",
    "ultralytics==8.0.232",
    "insightface==0.7.3",
    "gradio==4.8.0"
]

missing_deps = ["Cython", "onnx>=1.15.0"]

total_operations = (1 +  # Clear cache
                   len(problematic_packages) +  # Uninstalls
                   len(foundation_packages) +  # Foundation installs
                   len(dependency_packages) +  # Dependency installs
                   len(missing_deps) +  # Missing deps
                   1)  # Verify constraints

# Initialize package tracking
initialize_package_list(["Clear pip cache"], "System")
initialize_package_list(problematic_packages, "Uninstall")
initialize_package_list(foundation_packages, "Install")
initialize_package_list(dependency_packages, "Install")
initialize_package_list(missing_deps, "Install")
initialize_package_list(["Verify constraints"], "System")

# Display initial table
display(HTML('<div class="subtitle">📋 Package Installation Progress</div>'))
update_package_table()

# Start the process
show_status("🚀 Starting strict version pinning package fix...", "info")
update_progress(0, "Initializing...")
current_operation = 0

# Step 1: Clear pip cache
current_operation += 1
progress_percent = int((current_operation / total_operations) * 100)
update_progress(progress_percent, f"Step {current_operation}/{total_operations}: Clearing pip cache")
run_command_with_progress([sys.executable, '-m', 'pip', 'cache', 'purge'], "Clear pip cache", timeout=60)

# Step 2: Uninstall problematic packages
for package in problematic_packages:
    current_operation += 1
    progress_percent = int((current_operation / total_operations) * 100)
    update_progress(progress_percent, f"Step {current_operation}/{total_operations}: Uninstalling {package}")
    run_command_with_progress([sys.executable, '-m', 'pip', 'uninstall', '-y', package], f"Uninstall {package}", timeout=120)

# Step 3: Install foundation packages with constraints
for package in foundation_packages:
    current_operation += 1
    progress_percent = int((current_operation / total_operations) * 100)
    update_progress(progress_percent, f"Step {current_operation}/{total_operations}: Installing {package}")

    # Special handling for PyTorch packages
    if "torch" in package and "cu121" in package:
        if package == "torch==2.1.2+cu121":
            cmd = [sys.executable, '-m', 'pip', 'install', '--upgrade-strategy', 'only-if-needed']
            cmd.extend(['--index-url', 'https://download.pytorch.org/whl/cu121'])
            cmd.extend(['--constraint', constraints_file])
            cmd.extend(['torch==2.1.2+cu121', '--force-reinstall', '--no-cache-dir'])
        elif package == "torchvision==0.16.2+cu121":
            cmd = [sys.executable, '-m', 'pip', 'install', '--upgrade-strategy', 'only-if-needed']
            cmd.extend(['--index-url', 'https://download.pytorch.org/whl/cu121'])
            cmd.extend(['--constraint', constraints_file])
            cmd.extend(['torchvision==0.16.2+cu121', '--force-reinstall', '--no-cache-dir'])
        elif package == "torchaudio==2.1.2+cu121":
            cmd = [sys.executable, '-m', 'pip', 'install', '--upgrade-strategy', 'only-if-needed']
            cmd.extend(['--index-url', 'https://download.pytorch.org/whl/cu121'])
            cmd.extend(['--constraint', constraints_file])
            cmd.extend(['torchaudio==2.1.2+cu121', '--force-reinstall', '--no-cache-dir'])
        else:
            cmd = [sys.executable, '-m', 'pip', 'install', '--constraint', constraints_file,
                   package, '--force-reinstall', '--no-cache-dir']
    else:
        cmd = [sys.executable, '-m', 'pip', 'install', '--constraint', constraints_file,
               package, '--force-reinstall', '--no-cache-dir']

    run_command_with_progress(cmd, f"Install {package}", timeout=600)

# Step 4: Install dependencies with constraints
for package in dependency_packages:
    current_operation += 1
    progress_percent = int((current_operation / total_operations) * 100)
    update_progress(progress_percent, f"Step {current_operation}/{total_operations}: Installing {package}")

    # Use constraints for all installations
    cmd = [sys.executable, '-m', 'pip', 'install', '--constraint', constraints_file,
           package, '--force-reinstall', '--no-cache-dir']

    run_command_with_progress(cmd, f"Install {package}", timeout=600)

# Step 5: Install missing dependencies with constraints
for dep in missing_deps:
    current_operation += 1
    progress_percent = int((current_operation / total_operations) * 100)
    update_progress(progress_percent, f"Step {current_operation}/{total_operations}: Installing {dep}")
    cmd = [sys.executable, '-m', 'pip', 'install', '--constraint', constraints_file,
           dep, '--force-reinstall', '--no-cache-dir']
    run_command_with_progress(cmd, f"Install {dep}", timeout=300)

# Step 6: Verify constraints are active
current_operation += 1
progress_percent = int((current_operation / total_operations) * 100)
update_progress(progress_percent, f"Step {current_operation}/{total_operations}: Verifying constraints")

# Check if constraints file is being used
show_status("Verifying that constraints are active...", "info")
try:
    result = subprocess.run([sys.executable, '-m', 'pip', 'list'], capture_output=True, text=True)
    if result.returncode == 0:
        show_status("✅ Constraints verification completed", "success")
        # Update package results
        for result in package_results:
            if result['package'] == "System: Verify constraints":
                result['status'] = 'success'
                result['details'] = 'Constraints are active'
                break
    else:
        show_status("❌ Constraints verification failed", "error")
        for result in package_results:
            if result['package'] == "System: Verify constraints":
                result['status'] = 'error'
                result['details'] = 'Verification failed'
                break
except Exception as e:
    show_status(f"❌ Error verifying constraints: {e}", "error")
    for result in package_results:
        if result['package'] == "System: Verify constraints":
            result['status'] = 'error'
            result['details'] = f"Error: {str(e)}"
            break

update_package_table()

# Final progress update
update_progress(100, "Strict version pinning package fix completed!")

# Summary statistics
success_count = len([r for r in package_results if r['status'] == 'success'])
warning_count = len([r for r in package_results if r['status'] == 'warning'])
error_count = len([r for r in package_results if r['status'] == 'error'])
total_count = len(package_results)

# Display final summary
display(HTML('<div class="subtitle">📊 Installation Summary</div>'))
summary_html = f"""
<div style="display: flex; justify-content: space-around; flex-wrap: wrap; margin: 20px 0;">
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 15px; padding: 20px; margin: 10px; text-align: center; min-width: 150px;">
        <div style="font-size: 28px; font-weight: 700;">{total_count}</div>
        <div style="font-size: 14px; opacity: 0.9;">TOTAL</div>
    </div>
    <div style="background: linear-gradient(135deg, #11998e 0%, #38ef7d 100%); color: white; border-radius: 15px; padding: 20px; margin: 10px; text-align: center; min-width: 150px;">
        <div style="font-size: 28px; font-weight: 700;">{success_count}</div>
        <div style="font-size: 14px; opacity: 0.9;">SUCCESS</div>
    </div>
    <div style="background: linear-gradient(135deg, #fc4a1a 0%, #f7b733 100%); color: white; border-radius: 15px; padding: 20px; margin: 10px; text-align: center; min-width: 150px;">
        <div style="font-size: 28px; font-weight: 700;">{warning_count}</div>
        <div style="font-size: 14px; opacity: 0.9;">TIMEOUTS</div>
    </div>
    <div style="background: linear-gradient(135deg, #eb3349 0%, #f45c43 100%); color: white; border-radius: 15px; padding: 20px; margin: 10px; text-align: center; min-width: 150px;">
        <div style="font-size: 28px; font-weight: 700;">{error_count}</div>
        <div style="font-size: 14px; opacity: 0.9;">ERRORS</div>
    </div>
</div>
"""
display(HTML(summary_html))

# Recommendations
if error_count > 0:
    show_status("🚨 Some packages failed to install. Check the table above for details.", "error")
elif warning_count > 0:
    show_status("⚠️ Some packages timed out. You may want to retry them individually.", "warning")
else:
    show_status("🎉 All packages installed successfully with strict version pinning!", "success")

show_status("🔄 Next steps:", "info")
show_status("1. Restart the Colab runtime (Runtime → Restart runtime)", "info")
show_status("2. Run the verification script again", "info")
show_status("3. All packages should now be pinned to the correct versions", "info")

# Display constraints file info
display(HTML('<div class="subtitle">📋 Constraints File</div>'))
display(HTML(f'<div class="info">Constraints file created at: {constraints_file}</div>'))
display(HTML('<div class="info">This file will prevent any package upgrades in future installations</div>'))

display(HTML('</div>'))
display(HTML('</div>'))

print("\n" + "="*80)
print("STRICT VERSION PINNING PACKAGE FIX COMPLETED")
print("="*80)
print(f"Total Operations: {total_count}")
print(f"Successful: {success_count}")
print(f"Timeouts: {warning_count}")
print(f"Errors: {error_count}")
print(f"\n📋 Constraints file: {constraints_file}")
print("\n🔄 Next Steps:")
print("1. Restart the Colab runtime (Runtime → Restart runtime)")
print("2. Run the verification script again")
print("3. All packages should now be pinned to the correct versions")
print("="*80)

In [ ]:
# ===== TARGETED PACKAGE FIX FOR MISSING PACKAGES =====
from IPython.display import HTML, display, clear_output
import subprocess
import sys
import os
import time
import threading
from datetime import datetime

# Enhanced UI Styles
CSS = """
<style>
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700&display=swap');
body {
    font-family: 'Inter', sans-serif;
    background: linear-gradient(135deg, #f5f7fa 0%, #e4e8f0 100%);
    margin: 0;
    padding: 0;
}
.container {
    max-width: 1200px;
    margin: 0 auto;
    padding: 20px;
}
.title {
    color: #2c3e50;
    font-size: 32px;
    font-weight: 700;
    margin-bottom: 25px;
    text-align: center;
    background: linear-gradient(90deg, #e74c3c, #f39c12, #3498db);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
}
.subtitle {
    color: #34495e;
    font-size: 22px;
    font-weight: 600;
    margin-top: 25px;
    margin-bottom: 15px;
    padding-bottom: 8px;
    border-bottom: 3px solid #e74c3c;
}
.card {
    background: white;
    border-radius: 15px;
    padding: 25px;
    margin-bottom: 25px;
    box-shadow: 0 8px 25px rgba(0,0,0,0.08);
}
.info {
    color: #34495e;
    margin-bottom: 12px;
    font-size: 16px;
    line-height: 1.6;
}
.success {
    color: #27ae60;
    font-weight: 600;
    font-size: 16px;
}
.error {
    color: #e74c3c;
    font-weight: 600;
    font-size: 16px;
}
.warning {
    color: #f39c12;
    font-weight: 600;
    font-size: 16px;
}
.progress-container {
    background: #ecf0f1;
    border-radius: 12px;
    height: 30px;
    margin: 20px 0;
    overflow: hidden;
    position: relative;
}
.progress-bar {
    background: linear-gradient(90deg, #e74c3c, #f39c12);
    height: 100%;
    width: 0%;
    border-radius: 12px;
    transition: width 0.5s ease;
    display: flex;
    align-items: center;
    justify-content: center;
    color: white;
    font-weight: 600;
    font-size: 16px;
}
.status {
    color: #2c3e50;
    font-weight: 600;
    margin-top: 15px;
    text-align: center;
    font-size: 20px;
}
.package-table {
    width: 100%;
    border-collapse: collapse;
    margin: 20px 0;
    background: white;
    border-radius: 10px;
    overflow: hidden;
    box-shadow: 0 4px 15px rgba(0,0,0,0.05);
}
.package-table th {
    background: linear-gradient(90deg, #e74c3c, #f39c12);
    color: white;
    padding: 15px;
    text-align: left;
    font-weight: 600;
    font-size: 16px;
}
.package-table td {
    padding: 12px 15px;
    border-bottom: 1px solid #ecf0f1;
    font-size: 15px;
}
.package-table tr:last-child td {
    border-bottom: none;
}
.status-badge {
    padding: 6px 12px;
    border-radius: 20px;
    font-size: 13px;
    font-weight: 600;
    text-transform: uppercase;
}
.status-success {
    background-color: #d4edda;
    color: #155724;
}
.status-error {
    background-color: #f8d7da;
    color: #721c24;
}
.status-warning {
    background-color: #fff3cd;
    color: #856404;
}
.status-pending {
    background-color: #d1ecf1;
    color: #0c5460;
}
.status-running {
    background-color: #e2e3e5;
    color: #383d41;
}
.spinner {
    display: inline-block;
    width: 16px;
    height: 16px;
    border: 2px solid #f3f3f3;
    border-top: 2px solid #e74c3c;
    border-radius: 50%;
    animation: spin 1s linear infinite;
    margin-right: 8px;
}
@keyframes spin {
    0% { transform: rotate(0deg); }
    100% { transform: rotate(360deg); }
}
</style>
"""

# Initialize
display(HTML(CSS))
display(HTML('<div class="container">'))
display(HTML('<div class="title">🎯 TARGETED PACKAGE FIX FOR MISSING PACKAGES</div>'))
display(HTML('<div class="card">'))

# Global variables for tracking
package_results = []
current_operation = 0
total_operations = 0
progress_lock = threading.Lock()

def update_progress(percent, status_text=""):
    progress_html = f"""
    <div class="progress-container">
        <div class="progress-bar" style="width: {percent}%">
            {percent}%
        </div>
    </div>
    <div class="status">{status_text}</div>
    """
    display(HTML(progress_html))

def show_status(message, status_type="info"):
    if status_type == "success":
        display(HTML(f'<div class="success">✓ {message}</div>'))
    elif status_type == "error":
        display(HTML(f'<div class="error">✗ {message}</div>'))
    elif status_type == "warning":
        display(HTML(f'<div class="warning">⚠ {message}</div>'))
    else:
        display(HTML(f'<div class="info">• {message}</div>'))

def update_package_table():
    table_html = """
    <table class="package-table">
        <thead>
            <tr>
                <th>Package</th>
                <th>Status</th>
                <th>Details</th>
            </tr>
        </thead>
        <tbody>
    """
    for result in package_results:
        status_class = f"status-{result['status']}"
        status_text = result['status'].title()
        if result['status'] == 'running':
            status_text = '<span class="spinner"></span>Installing'

        table_html += f"""
            <tr>
                <td><strong>{result['package']}</strong></td>
                <td><span class="status-badge {status_class}">{status_text}</span></td>
                <td>{result['details']}</td>
            </tr>
        """
    table_html += """
        </tbody>
    </table>
    """
    display(HTML(table_html))

def run_command_with_progress(cmd, description, timeout=600):
    global current_operation, package_results

    # Update status to running
    with progress_lock:
        for result in package_results:
            if result['package'] == description:
                result['status'] = 'running'
                result['details'] = 'Installation in progress...'
                break
        update_package_table()

    show_status(f"🔄 Processing: {description}", "info")

    try:
        # Start a timer thread for progress updates
        def progress_updater():
            elapsed = 0
            while elapsed < timeout:
                time.sleep(5)
                elapsed += 5
                with progress_lock:
                    for result in package_results:
                        if result['package'] == description and result['status'] == 'running':
                            result['details'] = f'Installing... ({elapsed}s elapsed)'
                            update_package_table()
                if elapsed >= timeout:
                    break

        progress_thread = threading.Thread(target=progress_updater)
        progress_thread.daemon = True
        progress_thread.start()

        # Run the command
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=timeout)

        # Update the result
        with progress_lock:
            for res in package_results:
                if res['package'] == description:
                    if result.returncode == 0:
                        res['status'] = 'success'
                        res['details'] = 'Successfully installed'
                        show_status(f"✅ Success: {description}", "success")
                    else:
                        res['status'] = 'error'
                        res['details'] = f"Error: {result.stderr[:200]}..."
                        show_status(f"❌ Failed: {description}", "error")
                    break
            update_package_table()

        return result.returncode == 0

    except subprocess.TimeoutExpired:
        with progress_lock:
            for res in package_results:
                if res['package'] == description:
                    res['status'] = 'warning'
                    res['details'] = f'Timeout after {timeout}s'
                    break
            update_package_table()
        show_status(f"⏱️ Timeout: {description}", "warning")
        return False
    except Exception as e:
        with progress_lock:
            for res in package_results:
                if res['package'] == description:
                    res['status'] = 'error'
                    res['details'] = f"Exception: {str(e)}"
                    break
            update_package_table()
        show_status(f"💥 Exception: {description} - {str(e)}", "error")
        return False

def initialize_package_list(packages, operation_type):
    global package_results
    for package in packages:
        package_results.append({
            'package': f"{operation_type}: {package}",
            'status': 'pending',
            'details': 'Waiting to process...'
        })

# First, fix the opencv-python version mismatch
show_status("🔧 Fixing opencv-python version mismatch...", "info")
run_command_with_progress([sys.executable, '-m', 'pip', 'uninstall', '-y', 'opencv-python'], "Uninstall opencv-python", timeout=120)
run_command_with_progress([sys.executable, '-m', 'pip', 'install', 'opencv-python==4.8.0.76', '--force-reinstall', '--no-cache-dir'], "Install opencv-python==4.8.0.76", timeout=300)

# Define missing packages to install
missing_packages = [
    "torchvision==0.16.2+cu121",
    "xformers==0.0.23",
    "scikit-image==0.22.0",
    "opencv-contrib-python==4.8.0.76",
    "diffusers==0.25.0",
    "ultralytics==8.0.232",
    "insightface==0.7.3",
    "spandrel==0.4.1"
]

total_operations = len(missing_packages)

# Initialize package tracking
initialize_package_list(missing_packages, "Install")

# Display initial table
display(HTML('<div class="subtitle">📋 Missing Packages Installation Progress</div>'))
update_package_table()

# Start the process
show_status("🚀 Installing missing packages...", "info")
update_progress(0, "Initializing...")
current_operation = 0

# Install missing packages with special handling
for package in missing_packages:
    current_operation += 1
    progress_percent = int((current_operation / total_operations) * 100)
    update_progress(progress_percent, f"Step {current_operation}/{total_operations}: Installing {package}")

    # Special handling for PyTorch-related packages
    if package == "torchvision==0.16.2+cu121":
        cmd = [sys.executable, '-m', 'pip', 'install', '--upgrade-strategy', 'only-if-needed']
        cmd.extend(['--index-url', 'https://download.pytorch.org/whl/cu121'])
        cmd.extend(['torchvision==0.16.2+cu121', '--force-reinstall', '--no-cache-dir'])
    elif package == "xformers==0.0.23":
        # Install xformers separately to avoid conflicts
        cmd = [sys.executable, '-m', 'pip', 'install', 'xformers==0.0.23', '--force-reinstall', '--no-cache-dir']
    elif package == "opencv-contrib-python==4.8.0.76":
        # Install opencv-contrib-python after opencv-python
        cmd = [sys.executable, '-m', 'pip', 'install', 'opencv-contrib-python==4.8.0.76', '--force-reinstall', '--no-cache-dir']
    elif package == "insightface==0.7.3":
        # Install insightface with --no-deps to avoid dependency conflicts
        cmd = [sys.executable, '-m', 'pip', 'install', 'insightface==0.7.3', '--no-deps', '--force-reinstall', '--no-cache-dir']
    else:
        cmd = [sys.executable, '-m', 'pip', 'install', package, '--force-reinstall', '--no-cache-dir']

    run_command_with_progress(cmd, f"Install {package}", timeout=600)

# Final progress update
update_progress(100, "Missing packages installation completed!")

# Summary statistics
success_count = len([r for r in package_results if r['status'] == 'success'])
warning_count = len([r for r in package_results if r['status'] == 'warning'])
error_count = len([r for r in package_results if r['status'] == 'error'])
total_count = len(package_results)

# Display final summary
display(HTML('<div class="subtitle">📊 Installation Summary</div>'))
summary_html = f"""
<div style="display: flex; justify-content: space-around; flex-wrap: wrap; margin: 20px 0;">
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 15px; padding: 20px; margin: 10px; text-align: center; min-width: 150px;">
        <div style="font-size: 28px; font-weight: 700;">{total_count}</div>
        <div style="font-size: 14px; opacity: 0.9;">TOTAL</div>
    </div>
    <div style="background: linear-gradient(135deg, #11998e 0%, #38ef7d 100%); color: white; border-radius: 15px; padding: 20px; margin: 10px; text-align: center; min-width: 150px;">
        <div style="font-size: 28px; font-weight: 700;">{success_count}</div>
        <div style="font-size: 14px; opacity: 0.9;">SUCCESS</div>
    </div>
    <div style="background: linear-gradient(135deg, #fc4a1a 0%, #f7b733 100%); color: white; border-radius: 15px; padding: 20px; margin: 10px; text-align: center; min-width: 150px;">
        <div style="font-size: 28px; font-weight: 700;">{warning_count}</div>
        <div style="font-size: 14px; opacity: 0.9;">TIMEOUTS</div>
    </div>
    <div style="background: linear-gradient(135deg, #eb3349 0%, #f45c43 100%); color: white; border-radius: 15px; padding: 20px; margin: 10px; text-align: center; min-width: 150px;">
        <div style="font-size: 28px; font-weight: 700;">{error_count}</div>
        <div style="font-size: 14px; opacity: 0.9;">ERRORS</div>
    </div>
</div>
"""
display(HTML(summary_html))

# Recommendations
if error_count > 0:
    show_status("🚨 Some packages failed to install. Check the table above for details.", "error")
elif warning_count > 0:
    show_status("⚠️ Some packages timed out. You may want to retry them individually.", "warning")
else:
    show_status("🎉 All missing packages installed successfully!", "success")

show_status("🔄 Next steps:", "info")
show_status("1. Restart the Colab runtime (Runtime → Restart runtime)", "info")
show_status("2. Run the verification script again", "info")
show_status("3. All packages should now be correctly installed", "info")

display(HTML('</div>'))
display(HTML('</div>'))

print("\n" + "="*80)
print("TARGETED PACKAGE FIX FOR MISSING PACKAGES COMPLETED")
print("="*80)
print(f"Total Operations: {total_count}")
print(f"Successful: {success_count}")
print(f"Timeouts: {warning_count}")
print(f"Errors: {error_count}")
print("\n🔄 Next Steps:")
print("1. Restart the Colab runtime (Runtime → Restart runtime)")
print("2. Run the verification script again")
print("3. All packages should now be correctly installed")
print("="*80)

In [ ]:
# ===== STEP 7: INSTALL NODE DEPENDENCIES =====
from IPython.display import HTML, display, clear_output
import subprocess
import sys
import os
import time
import importlib
import threading

# Reuse CSS from previous cell
display(HTML(CSS))

# Function to install package with version locking
def install_package(package_spec, force_reinstall=False):
    # Ensure package_spec is not None
    if package_spec is None:
        show_status("Package specification is None", "error")
        add_log_entry("Package specification is None", "error")
        return False

    # Extract package name
    if '==' in package_spec:
        package_name = package_spec.split('==')[0]
    elif '>=' in package_spec:
        package_name = package_spec.split('>=')[0]
    else:
        package_name = package_spec

    # Check if already installed with correct version
    if not force_reinstall:
        if '==' in package_spec:
            pkg_name, required_version = package_spec.split('==')
            try:
                result = subprocess.run([sys.executable, '-m', 'pip', 'show', pkg_name],
                                      capture_output=True, text=True)
                if result.returncode == 0:
                    for line in result.stdout.split('\n'):
                        if line.startswith('Version:'):
                            installed_version = line.split(':')[1].strip()
                            if installed_version == required_version:
                                show_status(f"Package already installed with correct version: {package_spec}", "info")
                                add_log_entry(f"Package already installed with correct version: {package_spec}", "info")
                                return True
            except Exception:
                pass
        elif is_package_installed(package_name):
            show_status(f"Package already installed: {package_name}", "info")
            add_log_entry(f"Package already installed: {package_name}", "info")
            return True

    # Install package with version locking
    cmd = [sys.executable, '-m', 'pip', 'install']

    # Add flags to prevent upgrades
    cmd.extend(['--upgrade-strategy', 'only-if-needed'])

    if force_reinstall:
        cmd.extend(['--force-reinstall', '--no-cache-dir'])

    cmd.append(package_spec)

    try:
        show_status(f"Installing: {package_spec}", "info")
        add_log_entry(f"Installing package: {package_spec}", "info")

        result = subprocess.run(cmd, capture_output=True, text=True)
        if result.returncode == 0:
            show_status(f"Successfully installed: {package_spec}", "success")
            add_log_entry(f"Successfully installed: {package_spec}", "success")
            return True
        else:
            show_status(f"Failed to install {package_spec}: {result.stderr}", "error")
            add_log_entry(f"Failed to install {package_spec}: {result.stderr}", "error")
            return False
    except Exception as e:
        show_status(f"Error installing {package_spec}: {str(e)}", "error")
        add_log_entry(f"Error installing {package_spec}: {str(e)}", "error")
        return False

# Function to check if package is installed
def is_package_installed(package_name):
    try:
        spec = importlib.util.find_spec(package_name)
        return spec is not None
    except Exception:
        return False

# Function to install package group
def install_package_group(category_name, packages):
    display(HTML(f'<div class="subtitle">{category_name}</div>'))
    add_log_entry(f"Installing package group: {category_name}", "info")

    installed = []
    skipped = []
    failed = []

    for package in packages:
        # Skip None packages
        if package is None:
            continue

        # Check if already installed with correct version
        if '==' in package:
            pkg_name = package.split('==')[0]
            required_version = package.split('==')[1]
        elif '>=' in package:
            pkg_name = package.split('>=')[0]
            required_version = None
        else:
            pkg_name = package
            required_version = None

        if is_package_installed(pkg_name):
            skipped.append(package)
            show_status(f"Skipping already installed package: {package}", "info")
            add_log_entry(f"Skipping already installed package: {package}", "info")
        else:
            # Install package
            success = install_package(package, force_reinstall=True)

            if success:
                installed.append(package)
            else:
                failed.append(package)

        time.sleep(0.3)  # Small delay for visual feedback

    return installed, skipped, failed

# Initialize
display(HTML('<div class="container">'))
display(HTML('<div class="title">Install Node Dependencies</div>'))
display(HTML('<div class="card">'))

update_progress(0, "Initializing node dependency installation...")
add_log_entry("Starting node dependency installation process", "info")

# Change to ComfyUI directory
try:
    os.chdir('/content/ComfyUI')
    show_status("Changed to ComfyUI directory", "success")
    add_log_entry("Changed to ComfyUI directory", "success")
except Exception as e:
    show_status(f"Failed to change directory: {str(e)}", "error")
    add_log_entry(f"Failed to change directory: {str(e)}", "error")

# Define package groups
package_groups = [
    {
        "name": "Custom Node Dependencies",
        "packages": [
            "ultralytics==8.0.232",
            "segment-anything==1.0",
            "groundingdino-py",
            "piexif==1.1.3",
            "blend-modes"
        ]
    },
    {
        "name": "ONNX Runtime GPU",
        "packages": [
            "onnxruntime-gpu==1.16.3"
        ]
    },
    {
        "name": "Protobuf Compatibility",
        "packages": [
            "protobuf==4.25.1"
        ]
    }
]

# Track installation results
all_installed = []
all_skipped = []
all_failed = []
total_groups = len(package_groups)

# Install package groups
for i, group in enumerate(package_groups):
    progress = int((i / total_groups) * 100)
    update_progress(progress, f"Installing {group['name']} packages...")
    add_log_entry(f"Installing package group: {group['name']}", "info")

    installed, skipped, failed = install_package_group(group['name'], group['packages'])
    all_installed.extend(installed)
    all_skipped.extend(skipped)
    all_failed.extend(failed)

    time.sleep(0.5)  # Small delay for visual feedback

# Final progress update
update_progress(100, "Node dependencies installation completed!")
add_log_entry("Node dependencies installation process completed", "success")

# Summary report
display(HTML('<div class="subtitle">Node Dependencies Installation Summary</div>'))
display(HTML(f'<div class="info">• Total package groups processed: {total_groups}</div>'))
display(HTML(f'<div class="info">• Successfully installed packages: {len(all_installed)}</div>'))
display(HTML(f'<div class="info">• Skipped (already installed): {len(all_skipped)}</div>'))
display(HTML(f'<div class="info">• Failed installations: {len(all_failed)}</div>'))

if all_failed:
    display(HTML('<div class="error">Some packages failed to install</div>'))
    for pkg in all_failed:
        display(HTML(f'<div class="error">- {pkg}</div>'))
else:
    display(HTML('<div class="success">All node dependencies installed successfully</div>'))

display(HTML('</div>'))  # Close card
display(HTML('</div>'))  # Close container

# Print summary report
print("\n" + "="*50)
print("SUMMARY REPORT")
print("="*50)
print(f"Status: Node Dependencies Installation")
print(f"Total Package Groups: {total_groups}")
print(f"Successfully Installed: {len(all_installed)}")
print(f"Skipped: {len(all_skipped)}")
print(f"Failed: {len(all_failed)}")

if all_installed:
    print("\nInstalled Packages:")
    for pkg in all_installed:
        print(f"  - {pkg}")

if all_skipped:
    print("\nSkipped Packages:")
    for pkg in all_skipped:
        print(f"  - {pkg}")

if all_failed:
    print("\nFailed Packages:")
    for pkg in all_failed:
        print(f"  - {pkg}")

print(f"\nNext Step: Download Main Checkpoints")
print("="*50)

In [ ]:
# ===== STEP 8: DOWNLOAD MAIN CHECKPOINTS =====
from IPython.display import HTML, display, clear_output
import subprocess
import sys
import os
import time
import requests
import re
from tqdm.auto import tqdm
import threading
from datetime import datetime

# Reuse CSS from previous cell
display(HTML(CSS))

# Function to remove existing file
def remove_existing_file(filepath):
    """Remove existing file to force re-download"""
    try:
        if os.path.exists(filepath):
            os.remove(filepath)
            show_status(f"Removed existing file: {os.path.basename(filepath)}", "info")
            add_log_entry(f"Removed existing file: {os.path.basename(filepath)}", "info")
            return True
    except Exception as e:
        show_status(f"Failed to remove file {filepath}: {str(e)}", "error")
        add_log_entry(f"Failed to remove file {filepath}: {str(e)}", "error")
    return False

# Function to get remote file size
def get_remote_file_size(url, timeout=30):
    """Get the expected file size from the remote server"""
    try:
        response = requests.head(url, timeout=timeout, allow_redirects=True)
        if response.status_code == 200:
            size = response.headers.get('content-length')
            if size:
                return int(size)
        return 0
    except Exception as e:
        add_log_entry(f"Failed to get remote file size for {url}: {str(e)}", "warning")
        return 0

# Function to monitor file size
def monitor_file_size(filepath, progress_bar, expected_size, stop_event):
    """Monitor file size and update progress bar"""
    while not stop_event.is_set():
        try:
            if os.path.exists(filepath):
                current_size = os.path.getsize(filepath)
                if expected_size > 0:
                    percentage = min(100, int((current_size / expected_size) * 100))
                    progress_bar.n = percentage
                else:
                    # If we don't know expected size, just show current size
                    progress_bar.set_description(f"📥 {os.path.basename(filepath)[:25]}... ({current_size/(1024*1024):.1f}MB)")
                progress_bar.refresh()
        except:
            pass
        time.sleep(0.5)

# Function to download with wget
def download_with_wget(url, output_path, max_retries=3):
    """Download file using wget with interactive progress bar"""
    show_status(f"Starting download: {output_path}", "info")
    add_log_entry(f"Starting download: {output_path}", "info")

    # Remove existing file to force re-download
    remove_existing_file(output_path)

    # Get expected file size for progress bar
    expected_size = get_remote_file_size(url)
    if expected_size > 0:
        show_status(f"Expected size: {expected_size/(1024*1024):.1f} MB", "info")
        add_log_entry(f"Expected size: {expected_size/(1024*1024):.1f} MB", "info")
    else:
        show_status("Could not determine expected size", "warning")
        add_log_entry("Could not determine expected size", "warning")

    start_time = time.time()
    for attempt in range(max_retries):
        try:
            show_status(f"Attempt {attempt + 1}/{max_retries}", "info")
            add_log_entry(f"Download attempt {attempt + 1}/{max_retries}", "info")

            # Create progress bar
            if expected_size > 0:
                progress_bar = tqdm(
                    total=100,
                    desc=f"📥 {os.path.basename(output_path)[:25]}...",
                    unit="%",
                    bar_format="{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt}"
                )
            else:
                progress_bar = tqdm(
                    desc=f"📥 {os.path.basename(output_path)[:25]}...",
                    unit="B",
                    unit_scale=True,
                    bar_format="{desc}: {n_fmt}"
                )

            # Set up monitoring
            stop_event = threading.Event()
            monitor_thread = threading.Thread(
                target=monitor_file_size,
                args=(output_path, progress_bar, expected_size, stop_event)
            )
            monitor_thread.daemon = True
            monitor_thread.start()

            # Start wget download
            cmd = [
                'wget',
                '--timeout=60',
                '--tries=1',  # We handle retries at our level
                '--no-check-certificate',
                '--user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36',
                '-O', output_path,
                '-q',  # Quiet mode to avoid interfering with our progress bar
                url
            ]

            process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

            # Wait for download to complete
            try:
                process.communicate(timeout=600)
                # Stop monitoring
                stop_event.set()
                monitor_thread.join(timeout=1)
                # Update progress bar to 100%
                if expected_size > 0:
                    progress_bar.n = 100
                progress_bar.close()

                if process.returncode == 0 and os.path.exists(output_path):
                    elapsed = time.time() - start_time
                    file_size = os.path.getsize(output_path)
                    speed_mb_s = (file_size / (1024*1024)) / elapsed if elapsed > 0 else 0
                    show_status(f"Download successful: {file_size/(1024*1024):.1f} MB in {elapsed:.1f}s ({speed_mb_s:.1f} MB/s)", "success")
                    add_log_entry(f"Download successful: {file_size/(1024*1024):.1f} MB in {elapsed:.1f}s ({speed_mb_s:.1f} MB/s)", "success")
                    return True
                else:
                    show_status(f"Download failed with return code: {process.returncode}", "error")
                    add_log_entry(f"Download failed with return code: {process.returncode}", "error")
            except subprocess.TimeoutExpired:
                process.kill()
                stop_event.set()
                monitor_thread.join(timeout=1)
                progress_bar.close()
                show_status(f"Timeout on attempt {attempt + 1}", "warning")
                add_log_entry(f"Timeout on attempt {attempt + 1}", "warning")
        except Exception as e:
            stop_event.set()
            if 'progress_bar' in locals():
                progress_bar.close()
            show_status(f"Error on attempt {attempt + 1}: {str(e)}", "error")
            add_log_entry(f"Error on attempt {attempt + 1}: {str(e)}", "error")

        if attempt < max_retries - 1:
            show_status("Waiting 5 seconds before retry...", "info")
            add_log_entry("Waiting 5 seconds before retry...", "info")
            time.sleep(5)

    return False

# Initialize
display(HTML('<div class="container">'))
display(HTML('<div class="title">Download Main Checkpoints</div>'))
display(HTML('<div class="card">'))

update_progress(0, "Initializing main checkpoints download...")
add_log_entry("Starting main checkpoints download process", "info")

# Change to checkpoints directory
try:
    os.makedirs('/content/ComfyUI_Models/checkpoints', exist_ok=True)
    os.chdir('/content/ComfyUI_Models/checkpoints')
    show_status("Changed to checkpoints directory", "success")
    add_log_entry("Changed to checkpoints directory", "success")
except Exception as e:
    show_status(f"Failed to change directory: {str(e)}", "error")
    add_log_entry(f"Failed to change directory: {str(e)}", "error")

# Define SDXL model downloads
sdxl_downloads = [
    (
        "https://huggingface.co/SG161222/RealVisXL_V4.0_Lightning/resolve/main/RealVisXL_V4.0_Lightning.safetensors",
        "RealVisXL_V4_Lightning.safetensors"
    ),
    (
        "https://huggingface.co/RunDiffusion/Juggernaut-XL-v9/resolve/main/Juggernaut-XL_v9_RunDiffusionPhoto_v2.safetensors",
        "JuggernautXL_v9.safetensors"
    ),
    (
        "https://civitai.com/api/download/models/1920523?type=Model&format=SafeTensor&size=pruned&fp=fp16",
        "epiCRealismXL.safetensors"
    )
]

# Download statistics
start_time = time.time()
successful_downloads = 0
total_size_gb = 0
results = []

# Process each download one by one
for i, (url, filename) in enumerate(sdxl_downloads):
    progress = int((i / len(sdxl_downloads)) * 100)
    update_progress(progress, f"Downloading {filename}")
    add_log_entry(f"Downloading {filename}", "info")

    # Download the file
    success = download_with_wget(url, filename)

    # Record results
    if success and os.path.exists(filename):
        successful_downloads += 1
        file_size = os.path.getsize(filename) / (1024**3)
        total_size_gb += file_size
        elapsed_time = time.time() - start_time
        speed_mb_s = (file_size * 1024) / elapsed_time if elapsed_time > 0 else 0
        results.append({
            'Filename': filename,
            'Status': '✅ Success',
            'Size (GB)': f"{file_size:.2f}",
            'Speed (MB/s)': f"{speed_mb_s:.1f}"
        })
        show_status(f"Successfully downloaded: {filename} ({file_size:.2f} GB)", "success")
        add_log_entry(f"Successfully downloaded: {filename} ({file_size:.2f} GB)", "success")
    else:
        results.append({
            'Filename': filename,
            'Status': '❌ Failed',
            'Size (GB)': '0.00',
            'Speed (MB/s)': '0.0'
        })
        show_status(f"Failed to download: {filename}", "error")
        add_log_entry(f"Failed to download: {filename}", "error")

# Final progress update
update_progress(100, "Main checkpoints download completed!")
add_log_entry("Main checkpoints download process completed", "success")

# Final summary
elapsed_time = time.time() - start_time
display(HTML('<div class="subtitle">Download Summary</div>'))
show_status(f"Successful: {successful_downloads}/{len(sdxl_downloads)}", "info")
show_status(f"Total size: {total_size_gb:.2f} GB", "info")
show_status(f"Total time: {elapsed_time:.1f} seconds", "info")
if total_size_gb > 0 and elapsed_time > 0:
    avg_speed_mbps = (total_size_gb * 8192) / elapsed_time
    show_status(f"Average speed: {avg_speed_mbps:.1f} Mbps", "info")

# Display results table
display(HTML('<div class="subtitle">Download Results</div>'))
for result in results:
    if '✅' in result['Status']:
        display(HTML(f'<div class="success">{result["Filename"]}: {result["Size (GB)"]} GB</div>'))
    else:
        display(HTML(f'<div class="error">{result["Filename"]}: Failed</div>'))

# Final verification
display(HTML('<div class="subtitle">Final Verification</div>'))
all_files_exist = True
for _, filename in sdxl_downloads:
    if os.path.exists(filename):
        size_gb = os.path.getsize(filename) / (1024**3)
        show_status(f"{filename}: {size_gb:.2f} GB", "success")
        add_log_entry(f"Verified {filename}: {size_gb:.2f} GB", "success")
    else:
        show_status(f"{filename}: Missing", "error")
        add_log_entry(f"Missing {filename}", "error")
        all_files_exist = False

if all_files_exist and successful_downloads == len(sdxl_downloads):
    show_status("ALL DOWNLOADS COMPLETED SUCCESSFULLY!", "success")
    add_log_entry("ALL DOWNLOADS COMPLETED SUCCESSFULLY!", "success")
    show_status("Ready for next step", "info")
    add_log_entry("Ready for next step", "info")
else:
    show_status("Some downloads failed. You may need to retry.", "error")
    add_log_entry("Some downloads failed. You may need to retry.", "error")

display(HTML('</div>'))  # Close card
display(HTML('</div>'))  # Close container

# Print summary report
print("\n" + "="*50)
print("SUMMARY REPORT")
print("="*50)
print(f"Status: Main Checkpoints Download")
print(f"Total Checkpoints: {len(sdxl_downloads)}")
print(f"Successfully Downloaded: {successful_downloads}")
print(f"Failed: {len(sdxl_downloads) - successful_downloads}")
print(f"Total Size: {total_size_gb:.2f} GB")
print(f"Total Time: {elapsed_time:.1f} seconds")

if successful_downloads > 0:
    print("\nDownloaded Checkpoints:")
    for _, filename in sdxl_downloads:
        if os.path.exists(filename):
            size_gb = os.path.getsize(filename) / (1024**3)
            print(f"  - {filename}: {size_gb:.2f} GB")

if len(sdxl_downloads) - successful_downloads > 0:
    print("\nFailed Downloads:")
    for _, filename in sdxl_downloads:
        if not os.path.exists(filename):
            print(f"  - {filename}: Missing")

print(f"\nNext Step: Download VAE and ControlNet Models")
print("="*50)

In [ ]:
# ===== STEP 9: DOWNLOAD VAE AND CONTROLNET MODELS =====
from IPython.display import HTML, display, clear_output
import subprocess
import sys
import os
import time
import requests
import re
from tqdm.auto import tqdm
import threading
from datetime import datetime

# Reuse CSS from previous cell
display(HTML(CSS))

# Function to remove existing file
def remove_existing_file(filepath):
    """Remove existing file to force re-download"""
    try:
        if os.path.exists(filepath):
            os.remove(filepath)
            show_status(f"Removed existing file: {os.path.basename(filepath)}", "info")
            add_log_entry(f"Removed existing file: {os.path.basename(filepath)}", "info")
            return True
    except Exception as e:
        show_status(f"Failed to remove file {filepath}: {str(e)}", "error")
        add_log_entry(f"Failed to remove file {filepath}: {str(e)}", "error")
    return False

# Function to get remote file size
def get_remote_file_size(url, timeout=30):
    """Get the expected file size from the remote server"""
    try:
        response = requests.head(url, timeout=timeout, allow_redirects=True)
        if response.status_code == 200:
            size = response.headers.get('content-length')
            if size:
                return int(size)
        return 0
    except Exception as e:
        add_log_entry(f"Failed to get remote file size for {url}: {str(e)}", "warning")
        return 0

# Function to monitor file size
def monitor_file_size(filepath, progress_bar, expected_size, stop_event):
    """Monitor file size and update progress bar"""
    while not stop_event.is_set():
        try:
            if os.path.exists(filepath):
                current_size = os.path.getsize(filepath)
                if expected_size > 0:
                    percentage = min(100, int((current_size / expected_size) * 100))
                    progress_bar.n = percentage
                else:
                    # If we don't know expected size, just show current size
                    progress_bar.set_description(f"📥 {os.path.basename(filepath)[:25]}... ({current_size/(1024*1024):.1f}MB)")
                progress_bar.refresh()
        except:
            pass
        time.sleep(0.5)

# Function to download with wget
def download_with_wget(url, output_path, max_retries=3):
    """Download file using wget with interactive progress bar"""
    show_status(f"Starting download: {output_path}", "info")
    add_log_entry(f"Starting download: {output_path}", "info")

    # Remove existing file to force re-download
    remove_existing_file(output_path)

    # Get expected file size for progress bar
    expected_size = get_remote_file_size(url)
    if expected_size > 0:
        show_status(f"Expected size: {expected_size/(1024*1024):.1f} MB", "info")
        add_log_entry(f"Expected size: {expected_size/(1024*1024):.1f} MB", "info")
    else:
        show_status("Could not determine expected size", "warning")
        add_log_entry("Could not determine expected size", "warning")

    start_time = time.time()
    for attempt in range(max_retries):
        try:
            show_status(f"Attempt {attempt + 1}/{max_retries}", "info")
            add_log_entry(f"Download attempt {attempt + 1}/{max_retries}", "info")

            # Create progress bar
            if expected_size > 0:
                progress_bar = tqdm(
                    total=100,
                    desc=f"📥 {os.path.basename(output_path)[:25]}...",
                    unit="%",
                    bar_format="{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt}"
                )
            else:
                progress_bar = tqdm(
                    desc=f"📥 {os.path.basename(output_path)[:25]}...",
                    unit="B",
                    unit_scale=True,
                    bar_format="{desc}: {n_fmt}"
                )

            # Set up monitoring
            stop_event = threading.Event()
            monitor_thread = threading.Thread(
                target=monitor_file_size,
                args=(output_path, progress_bar, expected_size, stop_event)
            )
            monitor_thread.daemon = True
            monitor_thread.start()

            # Start wget download
            cmd = [
                'wget',
                '--timeout=60',
                '--tries=1',  # We handle retries at our level
                '--no-check-certificate',
                '--user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36',
                '-O', output_path,
                '-q',  # Quiet mode to avoid interfering with our progress bar
                url
            ]

            process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

            # Wait for download to complete
            try:
                process.communicate(timeout=600)
                # Stop monitoring
                stop_event.set()
                monitor_thread.join(timeout=1)
                # Update progress bar to 100%
                if expected_size > 0:
                    progress_bar.n = 100
                progress_bar.close()

                if process.returncode == 0 and os.path.exists(output_path):
                    elapsed = time.time() - start_time
                    file_size = os.path.getsize(output_path)
                    speed_mb_s = (file_size / (1024*1024)) / elapsed if elapsed > 0 else 0
                    show_status(f"Download successful: {file_size/(1024*1024):.1f} MB in {elapsed:.1f}s ({speed_mb_s:.1f} MB/s)", "success")
                    add_log_entry(f"Download successful: {file_size/(1024*1024):.1f} MB in {elapsed:.1f}s ({speed_mb_s:.1f} MB/s)", "success")
                    return True
                else:
                    show_status(f"Download failed with return code: {process.returncode}", "error")
                    add_log_entry(f"Download failed with return code: {process.returncode}", "error")
            except subprocess.TimeoutExpired:
                process.kill()
                stop_event.set()
                monitor_thread.join(timeout=1)
                progress_bar.close()
                show_status(f"Timeout on attempt {attempt + 1}", "warning")
                add_log_entry(f"Timeout on attempt {attempt + 1}", "warning")
        except Exception as e:
            stop_event.set()
            if 'progress_bar' in locals():
                progress_bar.close()
            show_status(f"Error on attempt {attempt + 1}: {str(e)}", "error")
            add_log_entry(f"Error on attempt {attempt + 1}: {str(e)}", "error")

        if attempt < max_retries - 1:
            show_status("Waiting 5 seconds before retry...", "info")
            add_log_entry("Waiting 5 seconds before retry...", "info")
            time.sleep(5)

    return False

# Function to download model group
def download_model_group(category_name, models):
    display(HTML(f'<div class="subtitle">{category_name}</div>'))
    add_log_entry(f"Downloading model group: {category_name}", "info")

    downloaded = []
    skipped = []
    failed = []

    for model in models:
        # Ensure directory exists
        os.makedirs(os.path.dirname(model['file_path']), exist_ok=True)

        if os.path.exists(model['file_path']):
            skipped.append(model['description'])
            show_status(f"Skipping already downloaded: {model['description']}", "info")
            add_log_entry(f"Skipping already downloaded: {model['description']}", "info")
        else:
            # Change to the target directory
            target_dir = os.path.dirname(model['file_path'])
            os.chdir(target_dir)

            # Download file
            filename = os.path.basename(model['file_path'])
            success = download_with_wget(model['url'], filename)

            if success:
                downloaded.append(model['description'])
            else:
                failed.append(model['description'])

        time.sleep(0.5)  # Small delay for visual feedback

    return downloaded, skipped, failed

# Initialize
display(HTML('<div class="container">'))
display(HTML('<div class="title">Download VAE and ControlNet Models</div>'))
display(HTML('<div class="card">'))

update_progress(0, "Initializing VAE and ControlNet models download...")
add_log_entry("Starting VAE and ControlNet models download process", "info")

# Define model groups
model_groups = [
    {
        "name": "VAE Model",
        "models": [
            {
                "url": "https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors",
                "file_path": "/content/ComfyUI_Models/vae/sdxl_vae.safetensors",
                "description": "SDXL VAE Model"
            }
        ]
    },
    {
        "name": "ControlNet Models",
        "models": [
            {
                "url": "https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-canny-rank256.safetensors",
                "file_path": "/content/ComfyUI_Models/controlnet/control_sdxl_canny.safetensors",
                "description": "ControlNet Canny Model"
            },
            {
                "url": "https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-depth-rank256.safetensors",
                "file_path": "/content/ComfyUI_Models/controlnet/control_sdxl_depth.safetensors",
                "description": "ControlNet Depth Model"
            }
        ]
    }
]

# Track download results
all_downloaded = []
all_skipped = []
all_failed = []
total_groups = len(model_groups)

# Download model groups
for i, group in enumerate(model_groups):
    progress = int((i / total_groups) * 100)
    update_progress(progress, f"Downloading {group['name']}...")
    add_log_entry(f"Downloading model group: {group['name']}", "info")

    downloaded, skipped, failed = download_model_group(group['name'], group['models'])
    all_downloaded.extend(downloaded)
    all_skipped.extend(skipped)
    all_failed.extend(failed)

    time.sleep(0.5)  # Small delay for visual feedback

# Final progress update
update_progress(100, "VAE and ControlNet models download completed!")
add_log_entry("VAE and ControlNet models download process completed", "success")

# Display downloaded files info
display(HTML('<div class="subtitle">Downloaded Model Files</div>'))
for group in model_groups:
    for model in group['models']:
        file_path = model['file_path']
        if os.path.exists(file_path):
            file_size = os.path.getsize(file_path) / (1024 * 1024)
            display(HTML(f'<div class="success">✓ {os.path.basename(file_path)} - {file_size:.2f} MB</div>'))
            add_log_entry(f"Verified {os.path.basename(file_path)}: {file_size:.2f} MB", "success")
        else:
            display(HTML(f'<div class="error">✗ {os.path.basename(file_path)} - Not found</div>'))
            add_log_entry(f"Missing {os.path.basename(file_path)}", "error")

# Summary report
display(HTML('<div class="subtitle">VAE and ControlNet Models Download Summary</div>'))
display(HTML(f'<div class="info">• Total model groups: {total_groups}</div>'))
display(HTML(f'<div class="info">• Successfully downloaded models: {len(all_downloaded)}</div>'))
display(HTML(f'<div class="info">• Skipped (already downloaded): {len(all_skipped)}</div>'))
display(HTML(f'<div class="info">• Failed downloads: {len(all_failed)}</div>'))

if all_failed:
    display(HTML('<div class="error">Some models failed to download</div>'))
    for model in all_failed:
        display(HTML(f'<div class="error">- {model}</div>'))
else:
    display(HTML('<div class="success">All VAE and ControlNet models downloaded successfully</div>'))

display(HTML('</div>'))  # Close card
display(HTML('</div>'))  # Close container

# Print summary report
print("\n" + "="*50)
print("SUMMARY REPORT")
print("="*50)
print(f"Status: VAE and ControlNet Models Download")
print(f"Total Model Groups: {total_groups}")
print(f"Successfully Downloaded: {len(all_downloaded)}")
print(f"Skipped: {len(all_skipped)}")
print(f"Failed: {len(all_failed)}")

if all_downloaded:
    print("\nDownloaded Models:")
    for model in all_downloaded:
        print(f"  - {model}")

if all_skipped:
    print("\nSkipped Models:")
    for model in all_skipped:
        print(f"  - {model}")

if all_failed:
    print("\nFailed Downloads:")
    for model in all_failed:
        print(f"  - {model}")

print(f"\nNext Step: Download Upscaling and Face Detection Models")
print("="*50)

In [ ]:
# ===== STEP 10: DOWNLOAD UPSCALING AND FACE DETECTION MODELS =====
from IPython.display import HTML, display, clear_output
import subprocess
import sys
import os
import time
import requests
import re
from tqdm.auto import tqdm
import threading
from datetime import datetime

# Reuse CSS from previous cell
display(HTML(CSS))

# Function to remove existing file
def remove_existing_file(filepath):
    """Remove existing file to force re-download"""
    try:
        if os.path.exists(filepath):
            os.remove(filepath)
            show_status(f"Removed existing file: {os.path.basename(filepath)}", "info")
            add_log_entry(f"Removed existing file: {os.path.basename(filepath)}", "info")
            return True
    except Exception as e:
        show_status(f"Failed to remove file {filepath}: {str(e)}", "error")
        add_log_entry(f"Failed to remove file {filepath}: {str(e)}", "error")
    return False

# Function to get remote file size
def get_remote_file_size(url, timeout=30):
    """Get the expected file size from the remote server"""
    try:
        response = requests.head(url, timeout=timeout, allow_redirects=True)
        if response.status_code == 200:
            size = response.headers.get('content-length')
            if size:
                return int(size)
        return 0
    except Exception as e:
        add_log_entry(f"Failed to get remote file size for {url}: {str(e)}", "warning")
        return 0

# Function to monitor file size
def monitor_file_size(filepath, progress_bar, expected_size, stop_event):
    """Monitor file size and update progress bar"""
    while not stop_event.is_set():
        try:
            if os.path.exists(filepath):
                current_size = os.path.getsize(filepath)
                if expected_size > 0:
                    percentage = min(100, int((current_size / expected_size) * 100))
                    progress_bar.n = percentage
                else:
                    # If we don't know expected size, just show current size
                    progress_bar.set_description(f"📥 {os.path.basename(filepath)[:25]}... ({current_size/(1024*1024):.1f}MB)")
                progress_bar.refresh()
        except:
            pass
        time.sleep(0.5)

# Function to download with wget
def download_with_wget(url, output_path, max_retries=3):
    """Download file using wget with interactive progress bar"""
    show_status(f"Starting download: {output_path}", "info")
    add_log_entry(f"Starting download: {output_path}", "info")

    # Remove existing file to force re-download
    remove_existing_file(output_path)

    # Get expected file size for progress bar
    expected_size = get_remote_file_size(url)
    if expected_size > 0:
        show_status(f"Expected size: {expected_size/(1024*1024):.1f} MB", "info")
        add_log_entry(f"Expected size: {expected_size/(1024*1024):.1f} MB", "info")
    else:
        show_status("Could not determine expected size", "warning")
        add_log_entry("Could not determine expected size", "warning")

    start_time = time.time()
    for attempt in range(max_retries):
        try:
            show_status(f"Attempt {attempt + 1}/{max_retries}", "info")
            add_log_entry(f"Download attempt {attempt + 1}/{max_retries}", "info")

            # Create progress bar
            if expected_size > 0:
                progress_bar = tqdm(
                    total=100,
                    desc=f"📥 {os.path.basename(output_path)[:25]}...",
                    unit="%",
                    bar_format="{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt}"
                )
            else:
                progress_bar = tqdm(
                    desc=f"📥 {os.path.basename(output_path)[:25]}...",
                    unit="B",
                    unit_scale=True,
                    bar_format="{desc}: {n_fmt}"
                )

            # Set up monitoring
            stop_event = threading.Event()
            monitor_thread = threading.Thread(
                target=monitor_file_size,
                args=(output_path, progress_bar, expected_size, stop_event)
            )
            monitor_thread.daemon = True
            monitor_thread.start()

            # Start wget download
            cmd = [
                'wget',
                '--timeout=60',
                '--tries=1',  # We handle retries at our level
                '--no-check-certificate',
                '--user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36',
                '-O', output_path,
                '-q',  # Quiet mode to avoid interfering with our progress bar
                url
            ]

            process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

            # Wait for download to complete
            try:
                process.communicate(timeout=600)
                # Stop monitoring
                stop_event.set()
                monitor_thread.join(timeout=1)
                # Update progress bar to 100%
                if expected_size > 0:
                    progress_bar.n = 100
                progress_bar.close()

                if process.returncode == 0 and os.path.exists(output_path):
                    elapsed = time.time() - start_time
                    file_size = os.path.getsize(output_path)
                    speed_mb_s = (file_size / (1024*1024)) / elapsed if elapsed > 0 else 0
                    show_status(f"Download successful: {file_size/(1024*1024):.1f} MB in {elapsed:.1f}s ({speed_mb_s:.1f} MB/s)", "success")
                    add_log_entry(f"Download successful: {file_size/(1024*1024):.1f} MB in {elapsed:.1f}s ({speed_mb_s:.1f} MB/s)", "success")
                    return True
                else:
                    show_status(f"Download failed with return code: {process.returncode}", "error")
                    add_log_entry(f"Download failed with return code: {process.returncode}", "error")
            except subprocess.TimeoutExpired:
                process.kill()
                stop_event.set()
                monitor_thread.join(timeout=1)
                progress_bar.close()
                show_status(f"Timeout on attempt {attempt + 1}", "warning")
                add_log_entry(f"Timeout on attempt {attempt + 1}", "warning")
        except Exception as e:
            stop_event.set()
            if 'progress_bar' in locals():
                progress_bar.close()
            show_status(f"Error on attempt {attempt + 1}: {str(e)}", "error")
            add_log_entry(f"Error on attempt {attempt + 1}: {str(e)}", "error")

        if attempt < max_retries - 1:
            show_status("Waiting 5 seconds before retry...", "info")
            add_log_entry("Waiting 5 seconds before retry...", "info")
            time.sleep(5)

    return False

# Function to download model group
def download_model_group(category_name, models):
    display(HTML(f'<div class="subtitle">{category_name}</div>'))
    add_log_entry(f"Downloading model group: {category_name}", "info")

    downloaded = []
    skipped = []
    failed = []

    for model in models:
        # Ensure directory exists
        os.makedirs(os.path.dirname(model['file_path']), exist_ok=True)

        if os.path.exists(model['file_path']):
            skipped.append(model['description'])
            show_status(f"Skipping already downloaded: {model['description']}", "info")
            add_log_entry(f"Skipping already downloaded: {model['description']}", "info")
        else:
            # Change to the target directory
            target_dir = os.path.dirname(model['file_path'])
            os.chdir(target_dir)

            # Download file
            filename = os.path.basename(model['file_path'])
            success = download_with_wget(model['url'], filename)

            if success:
                downloaded.append(model['description'])
            else:
                failed.append(model['description'])

        time.sleep(0.5)  # Small delay for visual feedback

    return downloaded, skipped, failed

# Initialize
display(HTML('<div class="container">'))
display(HTML('<div class="title">Download Upscaling and Face Detection Models</div>'))
display(HTML('<div class="card">'))

update_progress(0, "Initializing upscaling and face detection models download...")
add_log_entry("Starting upscaling and face detection models download process", "info")

# Define model groups
model_groups = [
    {
        "name": "Upscaling Models",
        "models": [
            {
                "url": "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth",
                "file_path": "/content/ComfyUI_Models/upscale_models/RealESRGAN_x4plus.pth",
                "description": "Real-ESRGAN x4plus Upscaler"
            }
        ]
    },
    {
        "name": "Face Detection Models",
        "models": [
            {
                "url": "https://huggingface.co/Bingsu/adetailer/resolve/main/face_yolov8n.pt",
                "file_path": "/content/ComfyUI/models/ultralytics/bbox/face_yolov8n.pt",
                "description": "Face Detection YOLOv8n Model"
            },
            {
                "url": "https://huggingface.co/Bingsu/adetailer/resolve/main/face_yolov8s.pt",
                "file_path": "/content/ComfyUI/models/ultralytics/bbox/face_yolov8s.pt",
                "description": "Face Detection YOLOv8s Model"
            }
        ]
    }
]

# Track download results
all_downloaded = []
all_skipped = []
all_failed = []
total_groups = len(model_groups)

# Download model groups
for i, group in enumerate(model_groups):
    progress = int((i / total_groups) * 100)
    update_progress(progress, f"Downloading {group['name']}...")
    add_log_entry(f"Downloading model group: {group['name']}", "info")

    downloaded, skipped, failed = download_model_group(group['name'], group['models'])
    all_downloaded.extend(downloaded)
    all_skipped.extend(skipped)
    all_failed.extend(failed)

    time.sleep(0.5)  # Small delay for visual feedback

# Final progress update
update_progress(100, "Upscaling and face detection models download completed!")
add_log_entry("Upscaling and face detection models download process completed", "success")

# Display downloaded files info
display(HTML('<div class="subtitle">Downloaded Model Files</div>'))
for group in model_groups:
    for model in group['models']:
        file_path = model['file_path']
        if os.path.exists(file_path):
            file_size = os.path.getsize(file_path) / (1024 * 1024)
            display(HTML(f'<div class="success">✓ {os.path.basename(file_path)} - {file_size:.2f} MB</div>'))
            add_log_entry(f"Verified {os.path.basename(file_path)}: {file_size:.2f} MB", "success")
        else:
            display(HTML(f'<div class="error">✗ {os.path.basename(file_path)} - Not found</div>'))
            add_log_entry(f"Missing {os.path.basename(file_path)}", "error")

# Summary report
display(HTML('<div class="subtitle">Upscaling and Face Detection Models Download Summary</div>'))
display(HTML(f'<div class="info">• Total model groups: {total_groups}</div>'))
display(HTML(f'<div class="info">• Successfully downloaded models: {len(all_downloaded)}</div>'))
display(HTML(f'<div class="info">• Skipped (already downloaded): {len(all_skipped)}</div>'))
display(HTML(f'<div class="info">• Failed downloads: {len(all_failed)}</div>'))

if all_failed:
    display(HTML('<div class="error">Some models failed to download</div>'))
    for model in all_failed:
        display(HTML(f'<div class="error">- {model}</div>'))
else:
    display(HTML('<div class="success">All upscaling and face detection models downloaded successfully</div>'))

display(HTML('</div>'))  # Close card
display(HTML('</div>'))  # Close container

# Print summary report
print("\n" + "="*50)
print("SUMMARY REPORT")
print("="*50)
print(f"Status: Upscaling and Face Detection Models Download")
print(f"Total Model Groups: {total_groups}")
print(f"Successfully Downloaded: {len(all_downloaded)}")
print(f"Skipped: {len(all_skipped)}")
print(f"Failed: {len(all_failed)}")

if all_downloaded:
    print("\nDownloaded Models:")
    for model in all_downloaded:
        print(f"  - {model}")

if all_skipped:
    print("\nSkipped Models:")
    for model in all_skipped:
        print(f"  - {model}")

if all_failed:
    print("\nFailed Downloads:")
    for model in all_failed:
        print(f"  - {model}")

print(f"\nNext Step: Prepare LoRA Directories")
print("="*50)

In [ ]:
# ===== STEP 11: PREPARE LORA DIRECTORIES =====
from IPython.display import HTML, display, clear_output
import os
import time
import threading

# Reuse CSS from previous cell
display(HTML(CSS))

# Function to create directory
def create_directory(dir_path):
    if os.path.exists(dir_path):
        show_status(f"Directory already exists: {dir_path}", "info")
        add_log_entry(f"Directory already exists: {dir_path}", "info")
        return True

    try:
        os.makedirs(dir_path, exist_ok=True)
        show_status(f"Created directory: {dir_path}", "success")
        add_log_entry(f"Created directory: {dir_path}", "success")
        return True
    except Exception as e:
        show_status(f"Error creating directory {dir_path}: {str(e)}", "error")
        add_log_entry(f"Error creating directory {dir_path}: {str(e)}", "error")
        return False

# Initialize
display(HTML('<div class="container">'))
display(HTML('<div class="title">Prepare LoRA Directories</div>'))
display(HTML('<div class="card">'))

update_progress(0, "Initializing LoRA directory setup...")
add_log_entry("Starting LoRA directory setup process", "info")

# Create LoRA directories
lora_base_path = '/content/ComfyUI_Models/loras'
lora_directories = [
    'face_enhancement',
    'lighting_photography',
    'style_effects',
    'detail_boost'
]

total_dirs = len(lora_directories)
created_dirs = []
existing_dirs = []

# Create directories
for i, dir_name in enumerate(lora_directories):
    progress = int((i / total_dirs) * 100)
    update_progress(progress, f"Creating directory: {dir_name}")
    add_log_entry(f"Creating directory: {dir_name}", "info")

    dir_path = os.path.join(lora_base_path, dir_name)
    if create_directory(dir_path):
        if os.path.exists(dir_path):
            created_dirs.append(dir_path)
        else:
            existing_dirs.append(dir_path)

    time.sleep(0.3)  # Small delay for visual feedback

# Final progress update
update_progress(100, "LoRA directory setup completed!")
add_log_entry("LoRA directory setup process completed", "success")

# Display directory structure
display(HTML('<div class="subtitle">LoRA Directory Structure</div>'))
display(HTML('<div class="info">/content/ComfyUI_Models/loras/</div>'))
display(HTML('<div class="info">├── face_enhancement/</div>'))
display(HTML('<div class="info">├── lighting_photography/</div>'))
display(HTML('<div class="info">├── style_effects/</div>'))
display(HTML('<div class="info">└── detail_boost/</div>'))

# Display instructions for downloading LoRA models
display(HTML('<div class="subtitle">How to Download LoRA Models</div>'))
display(HTML('<div class="info">Use the following commands in separate cells to download LoRA models:</div>'))

display(HTML('<div class="subtitle">Face Enhancement LoRAs</div>'))
display(HTML('<div class="info">%cd /content/ComfyUI_Models/loras/face_enhancement</div>'))
display(HTML('<div class="info"># !aria2c -c "[URL]" -o "[filename].safetensors"</div>'))

display(HTML('<div class="subtitle">Lighting and Photography LoRAs</div>'))
display(HTML('<div class="info">%cd /content/ComfyUI_Models/loras/lighting_photography</div>'))
display(HTML('<div class="info"># !aria2c -c "[URL]" -o "[filename].safetensors"</div>'))

display(HTML('<div class="subtitle">Style Effects LoRAs</div>'))
display(HTML('<div class="info">%cd /content/ComfyUI_Models/loras/style_effects</div>'))
display(HTML('<div class="info"># !aria2c -c "[URL]" -o "[filename].safetensors"</div>'))

display(HTML('<div class="subtitle">Detail Boost LoRAs</div>'))
display(HTML('<div class="info">%cd /content/ComfyUI_Models/loras/detail_boost</div>'))
display(HTML('<div class="info"># !aria2c -c "[URL]" -o "[filename].safetensors"</div>'))

# Summary report
display(HTML('<div class="subtitle">LoRA Directories Setup Summary</div>'))
display(HTML(f'<div class="info">• Total LoRA directories: {total_dirs}</div>'))
display(HTML(f'<div class="info">• Created directories: {len(created_dirs)}</div>'))
display(HTML(f'<div class="info">• Existing directories: {len(existing_dirs)}</div>'))
display(HTML('<div class="success">LoRA directory structure is ready for model downloads</div>'))

display(HTML('</div>'))  # Close card
display(HTML('</div>'))  # Close container

# Print summary report
print("\n" + "="*50)
print("SUMMARY REPORT")
print("="*50)
print(f"Status: LoRA Directories Setup")
print(f"Total Directories: {total_dirs}")
print(f"Created: {len(created_dirs)}")
print(f"Already Existed: {len(existing_dirs)}")

if created_dirs:
    print("\nCreated Directories:")
    for dir_path in created_dirs:
        print(f"  - {dir_path}")

if existing_dirs:
    print("\nExisting Directories:")
    for dir_path in existing_dirs:
        print(f"  - {dir_path}")

print(f"\nNext Step: Setup Cloudflare Tunnel")
print("="*50)

In [ ]:
# ===== STEP 12: SETUP CLOUDFLARE TUNNEL =====
from IPython.display import HTML, display, clear_output
import subprocess
import sys
import os
import time
import requests
import threading

# Reuse CSS from previous cell
display(HTML(CSS))

# Function to execute command and show output
def execute_command(cmd, description):
    show_status(f"Executing: {' '.join(cmd)}", "info")
    add_log_entry(f"Executing command: {' '.join(cmd)}", "info")

    try:
        result = subprocess.run(cmd, capture_output=True, text=True)
        if result.returncode == 0:
            show_status(f"Successfully executed: {description}", "success")
            add_log_entry(f"Successfully executed: {description}", "success")
            return True, result.stdout
        else:
            show_status(f"Failed to execute: {description}", "error")
            add_log_entry(f"Failed to execute: {description}: {result.stderr}", "error")
            return False, result.stderr
    except Exception as e:
        show_status(f"Error executing {description}: {str(e)}", "error")
        add_log_entry(f"Error executing {description}: {str(e)}", "error")
        return False, str(e)

# Function to check if file exists
def file_exists(file_path):
    return os.path.exists(file_path) and os.path.isfile(file_path)

# Initialize
display(HTML('<div class="container">'))
display(HTML('<div class="title">Setup Cloudflare Tunnel</div>'))
display(HTML('<div class="card">'))

update_progress(0, "Initializing Cloudflare tunnel setup...")
add_log_entry("Starting Cloudflare tunnel setup process", "info")

# Track operations
successful_ops = []
failed_ops = []
total_steps = 3

# Step 1: Download cloudflared
update_progress(33, "Downloading cloudflared...")
add_log_entry("Downloading cloudflared", "info")
step = 1

cloudflared_url = "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64"
cloudflared_path = "/content/cloudflared-linux-amd64"

if file_exists(cloudflared_path):
    show_status("Cloudflared already downloaded", "info")
    add_log_entry("Cloudflared already downloaded", "info")
    successful_ops.append("Download cloudflared")
else:
    success, output = execute_command(
        ['wget', '-q', '-O', cloudflared_path, cloudflared_url],
        "Download cloudflared"
    )

    if success:
        successful_ops.append("Download cloudflared")
    else:
        failed_ops.append("Download cloudflared")

# Step 2: Make cloudflared executable
update_progress(66, "Making cloudflared executable...")
add_log_entry("Making cloudflared executable", "info")
step += 1

success, output = execute_command(
    ['chmod', '+x', cloudflared_path],
    "Make cloudflared executable"
)

if success:
    successful_ops.append("Make cloudflared executable")
else:
    failed_ops.append("Make cloudflared executable")

# Step 3: Move cloudflared to system path
update_progress(100, "Moving cloudflared to system path...")
add_log_entry("Moving cloudflared to system path", "info")
step += 1

system_path = "/usr/local/bin/cloudflared"
success, output = execute_command(
    ['mv', cloudflared_path, system_path],
    "Move cloudflared to system path"
)

if success:
    successful_ops.append("Move cloudflared to system path")
else:
    failed_ops.append("Move cloudflared to system path")

# Verify installation
try:
    result = subprocess.run(['cloudflared', '--version'], capture_output=True, text=True)
    if result.returncode == 0:
        version = result.stdout.strip()
        show_status(f"Cloudflared installed successfully: {version}", "success")
        add_log_entry(f"Cloudflared installed successfully: {version}", "success")
        successful_ops.append("Verify cloudflared installation")
    else:
        show_status("Failed to verify cloudflared installation", "error")
        add_log_entry("Failed to verify cloudflared installation", "error")
        failed_ops.append("Verify cloudflared installation")
except Exception as e:
    show_status(f"Error verifying cloudflared installation: {str(e)}", "error")
    add_log_entry(f"Error verifying cloudflared installation: {str(e)}", "error")
    failed_ops.append("Verify cloudflared installation")

# Display file info
display(HTML('<div class="subtitle">Cloudflared Installation</div>'))
if file_exists(system_path):
    display(HTML('<div class="success">✓ /usr/local/bin/cloudflared - Installed</div>'))
    add_log_entry("Verified /usr/local/bin/cloudflared exists", "success")
else:
    display(HTML('<div class="error">✗ /usr/local/bin/cloudflared - Not found</div>'))
    add_log_entry("/usr/local/bin/cloudflared not found", "error")

# Final progress update
update_progress(100, "Cloudflare tunnel setup completed!")
add_log_entry("Cloudflare tunnel setup process completed", "success")

# Summary report
display(HTML('<div class="subtitle">Cloudflare Tunnel Setup Summary</div>'))
display(HTML(f'<div class="info">• Total operations: {len(successful_ops) + len(failed_ops)}</div>'))
display(HTML(f'<div class="info">• Successful operations: {len(successful_ops)}</div>'))
display(HTML(f'<div class="info">• Failed operations: {len(failed_ops)}</div>'))

if failed_ops:
    display(HTML('<div class="error">Some operations failed</div>'))
    for op in failed_ops:
        display(HTML(f'<div class="error">- {op}</div>'))
else:
    display(HTML('<div class="success">All operations completed successfully</div>'))

display(HTML('</div>'))  # Close card
display(HTML('</div>'))  # Close container

# Print summary report
print("\n" + "="*50)
print("SUMMARY REPORT")
print("="*50)
print(f"Status: Cloudflare Tunnel Setup")
print(f"Total Operations: {len(successful_ops) + len(failed_ops)}")
print(f"Successful: {len(successful_ops)}")
print(f"Failed: {len(failed_ops)}")

if successful_ops:
    print("\nSuccessful Operations:")
    for op in successful_ops:
        print(f"  - {op}")

if failed_ops:
    print("\nFailed Operations:")
    for op in failed_ops:
        print(f"  - {op}")

print(f"\nNext Step: Apply GPU Optimizations")
print("="*50)

In [ ]:
# ===== STEP 13: APPLY GPU OPTIMIZATIONS =====
from IPython.display import HTML, display, clear_output
import os
import time
import torch
import threading

# Reuse CSS from previous cell
display(HTML(CSS))

# Function to set environment variable
def set_env_var(var_name, var_value):
    try:
        os.environ[var_name] = var_value
        show_status(f"Set environment variable: {var_name}", "success")
        add_log_entry(f"Set environment variable: {var_name}={var_value}", "success")
        return True
    except Exception as e:
        show_status(f"Failed to set environment variable {var_name}: {str(e)}", "error")
        add_log_entry(f"Failed to set environment variable {var_name}: {str(e)}", "error")
        return False

# Function to display environment variable
def display_env_var(var_name):
    var_value = os.environ.get(var_name, "Not set")
    display(HTML(f'<div class="subtitle">{var_name}</div>'))
    display(HTML(f'<div class="info">{var_value}</div>'))
    add_log_entry(f"Environment variable {var_name}={var_value}", "info")

# Initialize
display(HTML('<div class="container">'))
display(HTML('<div class="title">Apply GPU Optimizations</div>'))
display(HTML('<div class="card">'))

update_progress(0, "Initializing GPU optimizations...")
add_log_entry("Starting GPU optimizations process", "info")

# Track environment variables set
env_vars_set = []
env_vars_failed = []
total_vars = 4

# Set environment variables
env_vars = [
    {
        "name": "PYTORCH_CUDA_ALLOC_CONF",
        "value": "max_split_size_mb:512,expandable_segments:True",
        "description": "PyTorch CUDA memory allocation configuration"
    },
    {
        "name": "TORCH_CUDNN_V8_API_ENABLED",
        "value": "1",
        "description": "Enable cuDNN v8 API"
    },
    {
        "name": "CUDA_LAUNCH_BLOCKING",
        "value": "0",
        "description": "Disable CUDA launch blocking for better performance"
    },
    {
        "name": "PYTHONUNBUFFERED",
        "value": "1",
        "description": "Disable Python output buffering"
    }
]

# Set environment variables
for i, env_var in enumerate(env_vars):
    progress = int((i / total_vars) * 100)
    update_progress(progress, f"Setting {env_var['name']}...")
    add_log_entry(f"Setting environment variable: {env_var['name']}", "info")

    if set_env_var(env_var['name'], env_var['value']):
        env_vars_set.append(env_var['name'])
    else:
        env_vars_failed.append(env_var['name'])

    time.sleep(0.3)  # Small delay for visual feedback

# Final progress update
update_progress(100, "GPU optimizations applied!")
add_log_entry("GPU optimizations process completed", "success")

# Display environment variables
for env_var in env_vars:
    display_env_var(env_var['name'])

# Display GPU information
try:
    display(HTML('<div class="subtitle">GPU Information</div>'))
    add_log_entry("Retrieving GPU information", "info")

    if torch.cuda.is_available():
        display(HTML(f'<div class="success">CUDA Available: Yes</div>'))
        display(HTML(f'<div class="info">GPU Name: {torch.cuda.get_device_name(0)}</div>'))
        display(HTML(f'<div class="info">CUDA Version: {torch.version.cuda}</div>'))
        display(HTML(f'<div class="info">PyTorch Version: {torch.__version__}</div>'))

        add_log_entry(f"CUDA Available: Yes", "success")
        add_log_entry(f"GPU Name: {torch.cuda.get_device_name(0)}", "info")
        add_log_entry(f"CUDA Version: {torch.version.cuda}", "info")
        add_log_entry(f"PyTorch Version: {torch.__version__}", "info")
    else:
        display(HTML(f'<div class="error">CUDA Available: No</div>'))
        display(HTML(f'<div class="warning">Running on CPU</div>'))

        add_log_entry("CUDA Available: No", "error")
        add_log_entry("Running on CPU", "warning")
except Exception as e:
    show_status(f"Error getting GPU information: {str(e)}", "error")
    add_log_entry(f"Error getting GPU information: {str(e)}", "error")

# Summary report
display(HTML('<div class="subtitle">GPU Optimizations Summary</div>'))
display(HTML(f'<div class="info">• Total environment variables: {total_vars}</div>'))
display(HTML(f'<div class="info">• Successfully set: {len(env_vars_set)}</div>'))
display(HTML(f'<div class="info">• Failed to set: {len(env_vars_failed)}</div>'))

if env_vars_failed:
    display(HTML('<div class="error">Some environment variables failed to set</div>'))
    for var in env_vars_failed:
        display(HTML(f'<div class="error">- {var}</div>'))
else:
    display(HTML('<div class="success">All environment variables set successfully</div>'))

display(HTML('<div class="success">GPU optimizations are ready for ComfyUI launch</div>'))

display(HTML('</div>'))  # Close card
display(HTML('</div>'))  # Close container

# Print summary report
print("\n" + "="*50)
print("SUMMARY REPORT")
print("="*50)
print(f"Status: GPU Optimizations Applied")
print(f"Total Environment Variables: {total_vars}")
print(f"Successfully Set: {len(env_vars_set)}")
print(f"Failed: {len(env_vars_failed)}")

if env_vars_set:
    print("\nEnvironment Variables Set:")
    for var in env_vars_set:
        print(f"  - {var}: {os.environ.get(var)}")

if env_vars_failed:
    print("\nFailed Variables:")
    for var in env_vars_failed:
        print(f"  - {var}")

print(f"\nNext Step: Launch ComfyUI with Tunnel")
print("="*50)

In [ ]:
# ===== STEP 14: LAUNCH COMFYUI WITH TUNNEL =====
from IPython.display import HTML, display, clear_output
import subprocess
import time
import threading
import re
import requests
import os
import signal
import sys
from datetime import datetime

# Reuse CSS from previous cell
display(HTML(CSS))

# Initialize
display(HTML('<div class="container">'))
display(HTML('<div class="title">Launch ComfyUI with Tunnel</div>'))
display(HTML('<div class="card">'))

update_progress(0, "Initializing ComfyUI launch...")
add_log_entry("Starting ComfyUI launch process", "info")

# Change to ComfyUI directory
try:
    os.chdir('/content/ComfyUI')
    show_status("Changed to ComfyUI directory", "success")
    add_log_entry("Changed to ComfyUI directory", "success")
except Exception as e:
    show_status(f"Failed to change directory: {str(e)}", "error")
    add_log_entry(f"Failed to change directory: {str(e)}", "error")

# Kill any existing processes
update_progress(10, "Cleaning up existing processes...")
show_status("Cleaning up any existing processes...", "info")
add_log_entry("Cleaning up existing processes", "info")

try:
    subprocess.run(['pkill', '-f', 'python.*main.py'], check=False)
    subprocess.run(['pkill', '-f', 'cloudflared'], check=False)
    show_status("Killed existing processes", "success")
    add_log_entry("Killed existing processes", "success")
    time.sleep(3)
except Exception as e:
    show_status(f"Error killing processes: {str(e)}", "error")
    add_log_entry(f"Error killing processes: {str(e)}", "error")

# Start ComfyUI server
update_progress(30, "Starting ComfyUI server...")
show_status("Starting ComfyUI server...", "info")
add_log_entry("Starting ComfyUI server", "info")

try:
    comfyui_process = subprocess.Popen([
        'python', 'main.py',
        '--listen', '127.0.0.1',
        '--port', '8188',
        '--lowvram',                        # Essential for T4 GPU
        '--use-split-cross-attention',      # Memory optimization
        '--preview-method', 'auto',
        '--disable-metadata',
        '--dont-upcast-attention'
    ], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

    show_status("ComfyUI server process started", "success")
    add_log_entry("ComfyUI server process started", "success")
except Exception as e:
    show_status(f"Failed to start ComfyUI server: {str(e)}", "error")
    add_log_entry(f"Failed to start ComfyUI server: {str(e)}", "error")

# Wait for server to start
update_progress(50, "Waiting for server to start...")
show_status("Waiting for server to start...", "info")
add_log_entry("Waiting for server to start", "info")

server_ready = False
for i in range(120):
    if comfyui_process.poll() is not None:
        show_status("ComfyUI failed to start", "error")
        add_log_entry("ComfyUI failed to start", "error")
        break

    # Check server status
    try:
        response = requests.get('http://127.0.0.1:8188/system_stats', timeout=2)
        if response.status_code == 200:
            show_status("ComfyUI server is running!", "success")
            add_log_entry("ComfyUI server is running!", "success")
            server_ready = True
            break
    except:
        pass

    if i % 10 == 0:
        show_status(f"Still starting... ({i}/120)", "info")
        add_log_entry(f"Still starting... ({i}/120)", "info")

    time.sleep(1)

if not server_ready:
    show_status("Failed to start ComfyUI server", "error")
    add_log_entry("Failed to start ComfyUI server", "error")

# Start Cloudflare tunnel
update_progress(70, "Creating public tunnel...")
show_status("Creating public tunnel...", "info")
add_log_entry("Creating public tunnel", "info")

try:
    tunnel_process = subprocess.Popen([
        'cloudflared', 'tunnel',
        '--url', 'http://127.0.0.1:8188',
        '--no-autoupdate'
    ], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

    show_status("Cloudflare tunnel process started", "success")
    add_log_entry("Cloudflare tunnel process started", "success")
except Exception as e:
    show_status(f"Failed to start tunnel: {str(e)}", "error")
    add_log_entry(f"Failed to start tunnel: {str(e)}", "error")

# Extract tunnel URL
update_progress(90, "Extracting tunnel URL...")
show_status("Extracting tunnel URL...", "info")
add_log_entry("Extracting tunnel URL", "info")

public_url = None
url_pattern = r'https://[a-zA-Z0-9\-]+\.trycloudflare\.com'

for _ in range(60):
    line = tunnel_process.stdout.readline()
    if line and 'trycloudflare.com' in line:
        match = re.search(url_pattern, line)
        if match:
            public_url = match.group(0)
            show_status("Tunnel URL extracted successfully", "success")
            add_log_entry(f"Tunnel URL extracted: {public_url}", "success")
            break
    time.sleep(1)

# Final progress update
update_progress(100, "ComfyUI launch completed!")
add_log_entry("ComfyUI launch process completed", "success")

# Display URL if available
if public_url:
    display(HTML('<div class="subtitle">ComfyUI Access URL</div>'))
    display(HTML(f'<div class="success">ComfyUI is accessible at:</div>'))
    display(HTML(f'<div class="info">{public_url}</div>'))
    add_log_entry(f"ComfyUI is accessible at: {public_url}", "success")
else:
    show_status("Failed to extract tunnel URL", "error")
    add_log_entry("Failed to extract tunnel URL", "error")

# Summary report
display(HTML('<div class="subtitle">ComfyUI Launch Summary</div>'))
display(HTML(f'<div class="info">• ComfyUI server status: {"Running" if server_ready else "Failed"}</div>'))
display(HTML(f'<div class="info">• Cloudflare tunnel status: {"Running" if public_url else "Failed"}</div>'))

if server_ready and public_url:
    display(HTML('<div class="success">ComfyUI is ready for use</div>'))
    add_log_entry("ComfyUI is ready for use", "success")
else:
    display(HTML('<div class="error">Some components failed to start</div>'))
    add_log_entry("Some components failed to start", "error")

display(HTML('</div>'))  # Close card
display(HTML('</div>'))  # Close container

# Print summary report
print("\n" + "="*50)
print("SUMMARY REPORT")
print("="*50)
print(f"Status: ComfyUI Launch")
print(f"Server Status: {'Running' if server_ready else 'Failed'}")
print(f"Tunnel Status: {'Running' if public_url else 'Failed'}")

if server_ready:
    print("\nComfyUI server is running on:")
    print("  - Local: http://127.0.0.1:8188")

if public_url:
    print(f"\nPublic URL:")
    print(f"  - {public_url}")

print("\nNote: The server will continue running in the background.")
print("You can now use ComfyUI through the public URL.")
print("="*50)

In [ ]:
# ===== VERIFICATION AND TESTING =====
from IPython.display import HTML, display, clear_output
import os
import sys
import time
import torch
import requests
import importlib
import threading

# Reuse CSS from previous cell
display(HTML(CSS))

# Function to display test result
def display_test_result(test_name, result, details=""):
    if result == "pass":
        show_status(f"{test_name}: {details}", "success")
        add_log_entry(f"Test passed: {test_name}: {details}", "success")
    elif result == "fail":
        show_status(f"{test_name}: {details}", "error")
        add_log_entry(f"Test failed: {test_name}: {details}", "error")
    else:
        show_status(f"{test_name}: {details}", "warning")
        add_log_entry(f"Test warning: {test_name}: {details}", "warning")

# Initialize
display(HTML('<div class="container">'))
display(HTML('<div class="title">Verification and Testing</div>'))
display(HTML('<div class="card">'))

update_progress(0, "Initializing verification...")
add_log_entry("Starting verification process", "info")

# Track test results
passed_tests = 0
failed_tests = 0
warning_tests = 0
total_tests = 0

# Test 1: PyTorch CUDA
update_progress(10, "Testing PyTorch CUDA...")
add_log_entry("Testing PyTorch CUDA", "info")
total_tests += 1

try:
    import torch
    cuda_available = torch.cuda.is_available()
    if cuda_available:
        gpu_name = torch.cuda.get_device_name(0)
        display_test_result("PyTorch CUDA", "pass", f"Available - {gpu_name}")
        passed_tests += 1
    else:
        display_test_result("PyTorch CUDA", "fail", "Not available")
        failed_tests += 1
except Exception as e:
    display_test_result("PyTorch CUDA", "fail", f"Error: {str(e)}")
    failed_tests += 1

# Test 2: XFormers
update_progress(20, "Testing XFormers...")
add_log_entry("Testing XFormers", "info")
total_tests += 1

try:
    import xformers
    xformers_version = xformers.__version__
    display_test_result("XFormers", "pass", f"Version {xformers_version}")
    passed_tests += 1
except Exception as e:
    display_test_result("XFormers", "fail", f"Error: {str(e)}")
    failed_tests += 1

# Test 3: Key packages
update_progress(30, "Testing key packages...")
add_log_entry("Testing key packages", "info")
total_tests += 1

packages_to_check = [
    'numpy', 'scipy', 'transformers', 'diffusers',
    'opencv-python', 'pillow', 'ultralytics', 'insightface'
]
missing_packages = []

for package in packages_to_check:
    try:
        if package == 'opencv-python':
            import cv2
            version = cv2.__version__
        elif package == 'pillow':
            import PIL
            version = PIL.__version__
        else:
            module = __import__(package)
            version = getattr(module, '__version__', 'Unknown')
    except Exception:
        missing_packages.append(package)

if not missing_packages:
    display_test_result("Key Packages", "pass", "All packages installed")
    passed_tests += 1
else:
    display_test_result("Key Packages", "fail", f"Missing: {', '.join(missing_packages)}")
    failed_tests += 1

# Test 4: Directory structure
update_progress(40, "Testing directory structure...")
add_log_entry("Testing directory structure", "info")
total_tests += 1

required_dirs = [
    '/content/ComfyUI_Models/checkpoints',
    '/content/ComfyUI_Models/loras',
    '/content/ComfyUI_Models/controlnet',
    '/content/ComfyUI_Models/upscale_models',
    '/content/ComfyUI_Models/vae',
    '/content/ComfyUI_Outputs'
]
missing_dirs = [dir_path for dir_path in required_dirs if not os.path.exists(dir_path)]

if not missing_dirs:
    display_test_result("Directory Structure", "pass", "All directories exist")
    passed_tests += 1
else:
    display_test_result("Directory Structure", "fail", f"Missing: {', '.join(missing_dirs)}")
    failed_tests += 1

# Test 5: Symbolic links
update_progress(50, "Testing symbolic links...")
add_log_entry("Testing symbolic links", "info")
total_tests += 1

symlinks = [
    '/content/ComfyUI/models/checkpoints',
    '/content/ComfyUI/models/loras',
    '/content/ComfyUI/models/controlnet',
    '/content/ComfyUI/models/upscale_models',
    '/content/ComfyUI/models/vae',
    '/content/ComfyUI/output'
]
broken_symlinks = [link for link in symlinks if not (os.path.islink(link) and os.path.exists(link))]

if not broken_symlinks:
    display_test_result("Symbolic Links", "pass", "All symlinks are valid")
    passed_tests += 1
else:
    display_test_result("Symbolic Links", "fail", f"Broken: {', '.join(broken_symlinks)}")
    failed_tests += 1

# Test 6: Model files
update_progress(60, "Testing model files...")
add_log_entry("Testing model files", "info")
total_tests += 1

required_models = [
    '/content/ComfyUI_Models/checkpoints/RealVisXL_V4_Lightning.safetensors',
    '/content/ComfyUI_Models/checkpoints/JuggernautXL_v9.safetensors',
    '/content/ComfyUI_Models/checkpoints/epiCRealismXL.safetensors',
    '/content/ComfyUI_Models/vae/sdxl_vae.safetensors',
    '/content/ComfyUI_Models/controlnet/control_sdxl_canny.safetensors',
    '/content/ComfyUI_Models/upscale_models/RealESRGAN_x4plus.pth'
]
missing_models = [model for model in required_models if not os.path.exists(model)]

if not missing_models:
    display_test_result("Model Files", "pass", "All required models are present")
    passed_tests += 1
else:
    display_test_result("Model Files", "warning", f"Missing: {', '.join(missing_models)}")
    warning_tests += 1

# Test 7: Custom nodes
update_progress(70, "Testing custom nodes...")
add_log_entry("Testing custom nodes", "info")
total_tests += 1

required_nodes = [
    '/content/ComfyUI/custom_nodes/ComfyUI-Manager',
    '/content/ComfyUI/custom_nodes/comfyui_controlnet_aux',
    '/content/ComfyUI/custom_nodes/was-node-suite-comfyui',
    '/content/ComfyUI/custom_nodes/ComfyUI-Impact-Pack'
]
missing_nodes = [node for node in required_nodes if not os.path.exists(node)]

if not missing_nodes:
    display_test_result("Custom Nodes", "pass", "All required nodes are present")
    passed_tests += 1
else:
    display_test_result("Custom Nodes", "warning", f"Missing: {', '.join(missing_nodes)}")
    warning_tests += 1

# Test 8: Cloudflared
update_progress(80, "Testing cloudflared...")
add_log_entry("Testing cloudflared", "info")
total_tests += 1

try:
    result = subprocess.run(['cloudflared', '--version'], capture_output=True, text=True)
    if result.returncode == 0:
        version = result.stdout.strip()
        display_test_result("Cloudflared", "pass", f"Version {version}")
        passed_tests += 1
    else:
        display_test_result("Cloudflared", "fail", "Not working")
        failed_tests += 1
except Exception as e:
    display_test_result("Cloudflared", "fail", f"Error: {str(e)}")
    failed_tests += 1

# Test 9: ComfyUI server
update_progress(90, "Testing ComfyUI server...")
add_log_entry("Testing ComfyUI server", "info")
total_tests += 1

try:
    response = requests.get('http://127.0.0.1:8188/system_stats', timeout=2)
    if response.status_code == 200:
        display_test_result("ComfyUI Server", "pass", "Running and accessible")
        passed_tests += 1
    else:
        display_test_result("ComfyUI Server", "fail", f"Returned status code {response.status_code}")
        failed_tests += 1
except Exception as e:
    display_test_result("ComfyUI Server", "fail", f"Error: {str(e)}")
    failed_tests += 1

# Final progress update
update_progress(100, "Verification completed!")
add_log_entry("Verification process completed", "success")

# Calculate score
score = int((passed_tests / total_tests) * 100)
score_color = "success" if score >= 80 else "warning" if score >= 50 else "error"

# Display score
display(HTML('<div class="subtitle">Installation Score</div>'))
display(HTML(f'<div class="{score_color}">{score}%</div>'))
add_log_entry(f"Installation score: {score}%", "success" if score >= 80 else "warning" if score >= 50 else "error")

# Summary report
display(HTML('<div class="subtitle">Verification Summary</div>'))
display(HTML(f'<div class="info">• Total tests: {total_tests}</div>'))
display(HTML(f'<div class="info">• Passed: {passed_tests}</div>'))
display(HTML(f'<div class="info">• Failed: {failed_tests}</div>'))
display(HTML(f'<div class="info">• Warnings: {warning_tests}</div>'))

if score >= 80:
    display(HTML('<div class="success">Installation is in good condition</div>'))
    add_log_entry("Installation is in good condition", "success")
elif score >= 50:
    display(HTML('<div class="warning">Installation has some issues but should work</div>'))
    add_log_entry("Installation has some issues but should work", "warning")
else:
    display(HTML('<div class="error">Installation has serious issues and may not work properly</div>'))
    add_log_entry("Installation has serious issues and may not work properly", "error")

display(HTML('</div>'))  # Close card
display(HTML('</div>'))  # Close container

# Print summary report
print("\n" + "="*50)
print("VERIFICATION REPORT")
print("="*50)
print(f"Total Tests: {total_tests}")
print(f"Passed: {passed_tests}")
print(f"Failed: {failed_tests}")
print(f"Warnings: {warning_tests}")
print(f"Score: {score}%")

if failed_tests > 0:
    print("\nFailed Tests:")
    if not torch.cuda.is_available():
        print("  - PyTorch CUDA not available")
    if missing_packages:
        print(f"  - Missing packages: {', '.join(missing_packages)}")
    if missing_dirs:
        print(f"  - Missing directories: {', '.join(missing_dirs)}")
    if broken_symlinks:
        print(f"  - Broken symlinks: {', '.join(broken_symlinks)}")
    try:
        subprocess.run(['cloudflared', '--version'], check=True)
    except:
        print("  - Cloudflared not working")
    try:
        response = requests.get('http://127.0.0.1:8188/system_stats', timeout=2)
        if response.status_code != 200:
            print("  - ComfyUI server not accessible")
    except:
        print("  - ComfyUI server not accessible")

if warning_tests > 0:
    print("\nWarnings:")
    if missing_models:
        print(f"  - Missing models: {', '.join(missing_models)}")
    if missing_nodes:
        print(f"  - Missing custom nodes: {', '.join(missing_nodes)}")

print("\n" + "="*50)

In [ ]:
# ===== COMMON ERROR FIXES =====
from IPython.display import HTML, display, clear_output
import subprocess
import sys
import os
import time
import importlib
import threading

# Reuse CSS from previous cell
display(HTML(CSS))

# Initialize
display(HTML('<div class="container">'))
display(HTML('<div class="title">Common Error Fixes</div>'))
display(HTML('<div class="card">'))

show_status("This section provides common fixes for issues that may occur during or after installation.", "info")
add_log_entry("Displaying common error fixes", "info")

# Fix 1: NumPy Version Conflicts
display(HTML('<div class="subtitle">Fix 1: NumPy Version Conflicts</div>'))
display(HTML('<div class="info">Run this if you encounter NumPy errors:</div>'))
display(HTML('<div class="warning">!pip uninstall -y numpy</div>'))
display(HTML('<div class="warning">!pip install numpy==1.26.2 --no-cache-dir --force-reinstall</div>'))

# Fix 2: CUDA/PyTorch Mismatch
display(HTML('<div class="subtitle">Fix 2: CUDA/PyTorch Mismatch</div>'))
display(HTML('<div class="info">Run this if you encounter CUDA/PyTorch errors:</div>'))
display(HTML('<div class="warning">!pip uninstall -y torch torchvision torchaudio xformers</div>'))
display(HTML('<div class="warning">!pip install torch==2.1.2+cu121 torchvision==0.16.2+cu121 torchaudio==2.1.2+cu121 --index-url https://download.pytorch.org/whl/cu121 --force-reinstall</div>'))
display(HTML('<div class="warning">!pip install xformers==0.0.23 --no-deps --force-reinstall</div>'))

# Fix 3: ONNX Runtime GPU Issues
display(HTML('<div class="subtitle">Fix 3: ONNX Runtime GPU Issues</div>'))
display(HTML('<div class="info">Run this if you encounter ONNX errors:</div>'))
display(HTML('<div class="warning">!pip install onnxruntime-gpu==1.16.3 --force-reinstall</div>'))

# Fix 4: Import Module Errors
display(HTML('<div class="subtitle">Fix 4: Import Module Errors</div>'))
display(HTML('<div class="info">Run this if you encounter import errors:</div>'))
display(HTML('<div class="warning">import pathlib</div>'))
display(HTML('<div class="warning">impact_path = pathlib.Path("/content/ComfyUI/custom_nodes/ComfyUI-Impact-Pack")</div>'))
display(HTML('<div class="warning">(impact_path / "models" / "__init__.py").touch()</div>'))
display(HTML('<div class="warning">(impact_path / "custom_nodes" / "__init__.py").touch()</div>'))

# Fix 5: Memory Issues
display(HTML('<div class="subtitle">Fix 5: Memory Issues on T4</div>'))
display(HTML('<div class="info">If you encounter memory issues:</div>'))
display(HTML('<div class="info">• Always use --lowvram flag when launching ComfyUI</div>'))
display(HTML('<div class="info">• Enable --use-split-cross-attention flag</div>'))
display(HTML('<div class="info">• Set batch size to 1 in workflows</div>'))
display(HTML('<div class="info">• Use tiled sampling for large images</div>'))

# Fix 6: ComfyUI Server Not Starting
display(HTML('<div class="subtitle">Fix 6: ComfyUI Server Not Starting</div>'))
display(HTML('<div class="info">If ComfyUI server fails to start:</div>'))
display(HTML('<div class="warning">!pkill -f "python.*main.py"</div>'))
display(HTML('<div class="warning">!cd /content/ComfyUI</div>'))
display(HTML('<div class="warning">!python main.py --listen 127.0.0.1 --port 8188 --lowvram --use-split-cross-attention</div>'))

# Fix 7: Cloudflare Tunnel Not Working
display(HTML('<div class="subtitle">Fix 7: Cloudflare Tunnel Not Working</div>'))
display(HTML('<div class="info">If Cloudflare tunnel fails to work:</div>'))
display(HTML('<div class="warning">!pkill -f "cloudflared"</div>'))
display(HTML('<div class="warning">!wget -q -O cloudflared-linux-amd64 https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64</div>'))
display(HTML('<div class="warning">!chmod +x cloudflared-linux-amd64</div>'))
display(HTML('<div class="warning">!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared</div>'))
display(HTML('<div class="warning">!cloudflared tunnel --url http://127.0.0.1:8188 --no-autoupdate</div>'))

# Fix 8: Models Not Loading
display(HTML('<div class="subtitle">Fix 8: Models Not Loading</div>'))
display(HTML('<div class="info">If models are not loading:</div>'))
display(HTML('<div class="info">• Check that model files exist in the correct directories</div>'))
display(HTML('<div class="info">• Verify symbolic links are working correctly</div>'))
display(HTML('<div class="info">• Check file permissions</div>'))
display(HTML('<div class="warning">!ls -la /content/ComfyUI/models/</div>'))
display(HTML('<div class="warning">!ls -la /content/ComfyUI_Models/</div>'))

# Fix 9: Custom Nodes Not Working
display(HTML('<div class="subtitle">Fix 9: Custom Nodes Not Working</div>'))
display(HTML('<div class="info">If custom nodes are not working:</div>'))
display(HTML('<div class="info">• Check that custom nodes are installed correctly</div>'))
display(HTML('<div class="info">• Install missing dependencies for custom nodes</div>'))
display(HTML('<div class="warning">!cd /content/ComfyUI</div>'))
display(HTML('<div class="warning">!pip install -r requirements.txt</div>'))

# Fix 10: Complete Reinstall
display(HTML('<div class="subtitle">Fix 10: Complete Reinstall</div>'))
display(HTML('<div class="info">If all else fails, you can perform a complete reinstall:</div>'))
display(HTML('<div class="warning">!rm -rf /content/ComfyUI</div>'))
display(HTML('<div class="warning">!rm -rf /content/ComfyUI_Models</div>'))
display(HTML('<div class="warning">!rm -rf /content/ComfyUI_Outputs</div>'))
display(HTML('<div class="info">Then run all installation cells again from the beginning.</div>'))

# Summary
display(HTML('<div class="subtitle">Summary</div>'))
display(HTML('<div class="info">Copy and paste the specific fix you need into a new cell and run it.</div>'))
display(HTML('<div class="info">If you continue to experience issues, please check the error messages and search for solutions online.</div>'))
display(HTML('<div class="success">Common error fixes are now available for troubleshooting</div>'))

add_log_entry("Common error fixes displayed", "success")

display(HTML('</div>'))  # Close card
display(HTML('</div>'))  # Close container

# Print summary report
print("\n" + "="*50)
print("COMMON ERROR FIXES")
print("="*50)
print("This section provides common fixes for issues that may occur during or after installation.")
print("Copy and paste the specific fix you need into a new cell and run it.")
print("\nAvailable fixes:")
print("1. NumPy Version Conflicts")
print("2. CUDA/PyTorch Mismatch")
print("3. ONNX Runtime GPU Issues")
print("4. Import Module Errors")
print("5. Memory Issues on T4")
print("6. ComfyUI Server Not Starting")
print("7. Cloudflare Tunnel Not Working")
print("8. Models Not Loading")
print("9. Custom Nodes Not Working")
print("10. Complete Reinstall")
print("\nIf you continue to experience issues, please check the error messages and search for solutions online.")
print("="*50)